In [ ]:
import os, time
from google.colab import drive

drive.mount("/content/drive")

# --- CONFIG ---
ZIP_DRIVE     = "/content/drive/MyDrive/OccMLP/Dataset100k/finalDataset.zip"
RAM_PATH      = "/mnt/ramdisk"
RAM_SIZE      = "150G"
RAM_OUT_DIR   = f"{RAM_PATH}/finalDataset_unzipped"
DISK_OUT_DIR  = "/content/DataSet/finalDataset"   # dataset final sur disk local

# 1) Monter tmpfs RAM
!sudo mkdir -p {RAM_PATH}
if not os.path.ismount(RAM_PATH):
    !sudo mount -t tmpfs -o size={RAM_SIZE} tmpfs {RAM_PATH}

print("=== BEFORE ===")
!df -h /content | tail -n 1
!df -h {RAM_PATH} | tail -n 1

# 2) Unzip direct Drive -> RAM (pas de ZIP sur /content)
if not os.path.exists(ZIP_DRIVE):
    raise FileNotFoundError(ZIP_DRIVE)

!sudo rm -rf "{RAM_OUT_DIR}"
!mkdir -p "{RAM_OUT_DIR}"

print("\n📦 Unzip Drive -> RAM (sans ZIP local)...")
t0 = time.time()
!unzip -q "{ZIP_DRIVE}" -d "{RAM_OUT_DIR}"
print(f"✅ Unzip RAM terminé en {time.time()-t0:.1f}s")

print("\n=== AFTER UNZIP ===")
!df -h /content | tail -n 1
!df -h {RAM_PATH} | tail -n 1
!du -sh "{RAM_OUT_DIR}" | tail -n 1

# 3) Copier RAM -> disk final (tu peux aussi entraîner direct en RAM et ne pas copier)
print("\n💾 Copie RAM -> /content (dataset final)...")
!sudo rm -rf "{DISK_OUT_DIR}"
!sudo mkdir -p "{DISK_OUT_DIR}"

t0 = time.time()
!rsync -a --info=progress2 "{RAM_OUT_DIR}/" "{DISK_OUT_DIR}/"
print(f"✅ Copie disk terminée en {time.time()-t0:.1f}s")

print("\n=== FINAL ===")
!df -h /content | tail -n 1
!du -sh "{DISK_OUT_DIR}" | tail -n 1

# (Optionnel) Libérer la RAM
!sudo rm -rf "{RAM_OUT_DIR}"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
=== BEFORE ===
overlay         236G   39G  197G  17% /
tmpfs           150G     0  150G   0% /mnt/ramdisk

📦 Unzip Drive -> RAM (sans ZIP local)...


In [ ]:
import os
import sys
import torch
import gc
from pathlib import Path
# %%
from google.colab import drive
drive.mount('/content/drive')

# Chemins
DRIVE_PATH = '/content/drive/MyDrive/'
PROJECT_PATH = f'{DRIVE_PATH}/OccMLP/Dataset100k'
DATASET_ZIP = f'{PROJECT_PATH}/finalDataset.zip'
LOCAL_DATASET = '/content/dataset/'
LOCAL_MODEL = '/content/drive/MyDrive/'
# Créer les dossiers
os.makedirs(PROJECT_PATH, exist_ok=True)
os.makedirs(f'{PROJECT_PATH}/checkpoints', exist_ok=True)
os.makedirs(f'{PROJECT_PATH}/logs', exist_ok=True)


# ## 3. Décompression du dataset

# %%
import time

# Vérifier si le dataset existe déjà
if os.path.exists(LOCAL_DATASET):
    print(f"Dataset déjà présent dans {LOCAL_DATASET}")
    !du -sh {LOCAL_DATASET}
else:
    # Vérifier le fichier zip
    if os.path.exists(DATASET_ZIP):
        print(f"Décompression de {DATASET_ZIP}...")
        start_time = time.time()

        # Déterminer le type de fichier et décompresser
        if DATASET_ZIP.endswith('.zip'):
            !unzip -q {DATASET_ZIP} -d /content/DataSet
        elif DATASET_ZIP.endswith('.tar.gz') or DATASET_ZIP.endswith('.tgz'):
            !tar -xzf {DATASET_ZIP} -C /content/DataSet
        elif DATASET_ZIP.endswith('.tar'):
            !tar -xf {DATASET_ZIP} -C /content/DataSet
        else:
            print("Format non reconnu!")

        elapsed = time.time() - start_time
        print(f"Décompression terminée en {elapsed:.1f} secondes")

        # Vérifier le résultat
        !ls -la /content/
        if os.path.exists(LOCAL_DATASET):
            !du -sh {LOCAL_DATASET}
            # Compter les fichiers
            !echo "Nombre d'images:" && find {LOCAL_DATASET}/images -name "*.jpg" | wc -l
            !echo "Nombre de voxels:" && find {LOCAL_DATASET}/occupancy -name "*.npz" | wc -l
    else:
        print(f"❌ Fichier non trouvé: {DATASET_ZIP}")
        print("Veuillez uploader votre dataset compressé dans Drive/voxel_project/")

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import random
from pathlib import Path
import numpy as np
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

# -----------------------------
# Config FINAL (comme ton training)
# -----------------------------
NUM_CLASSES = 7
CLASS_NAMES = [
    "0 Empty",
    "1 Unknown",
    "2 Road",
    "3 Sidewalk",
    "4 Vegetation",
    "5 Vehicles",
    "6 Occupied",
]

# -----------------------------
# Config RAW (comme ton NPZ export)
# labs_raw ∈ {-2, -1, 1..22} => 24 classes
# -----------------------------
CARLA_22 = [
    (1,  "Building"),
    (2,  "Fence"),
    (3,  "Other"),
    (4,  "Pedestrian"),
    (5,  "Pole"),
    (6,  "RoadLine"),
    (7,  "Road"),
    (8,  "SideWalk"),
    (9,  "Vegetation"),
    (10, "Vehicles"),
    (11, "Wall"),
    (12, "TrafficSign"),
    (13, "Sky"),
    (14, "Ground"),
    (15, "Bridge"),
    (16, "RailTrack"),
    (17, "GuardRail"),
    (18, "TrafficLight"),
    (19, "Static"),
    (20, "Dynamic"),
    (21, "Water"),
    (22, "Terrain"),
]
RAW_CLASS_IDS = [-2, -1] + [cid for cid, _ in CARLA_22]
RAW_CLASS_NAMES = ["-2 Empty", "-1 Unknown"] + [f"{cid} {name}" for cid, name in CARLA_22]
RAW_NUM_CLASSES = len(RAW_CLASS_IDS)  # 24

# ROI identique à ton Dataset (__getitem__)
EPS = 5e-6
X_MIN, X_MAX = -15 + EPS, 15 - EPS
Y_MIN, Y_MAX = -15 + EPS, 15 - EPS
Z_MIN, Z_MAX = -2  + EPS,  6 - EPS

IGNORE_LABEL = 255

def map_labels_like_training(labs_raw: np.ndarray) -> np.ndarray:
    """
    Mapping identique à ton ImplicitOccDataset.__getitem__ :

    labs_raw ∈ { -2 (Empty), -1 (Unknown), 1..22 (CARLA) }
    Road=7, Sidewalk=8, Vegetation=9, Vehicles=10
    Le reste => Occupied (6)
    Ignore=255 reste ignore.

    returns: labels ∈ {0..6} ou 255
    """
    labs_raw = labs_raw.astype(np.int16, copy=False)
    labels = np.full_like(labs_raw, fill_value=IGNORE_LABEL, dtype=np.int16)

    valid = (labs_raw != IGNORE_LABEL)

    empty_mask   = valid & (labs_raw == -2)
    unknown_mask = valid & (labs_raw == -1)

    road_mask     = valid & (labs_raw == 1)
    sidewalk_mask = valid & (labs_raw == 2)
    veget_mask = valid & np.isin(labs_raw, [9, 10])
    vehicle_mask = valid & np.isin(labs_raw, [13, 14, 15, 16, 17, 18, 19])

    labels[empty_mask] = 0
    labels[unknown_mask] = 1
    labels[road_mask] = 2
    labels[sidewalk_mask] = 3
    labels[veget_mask] = 4
    labels[vehicle_mask] = 5

    known = empty_mask | unknown_mask | road_mask | sidewalk_mask | veget_mask | vehicle_mask
    other_occupied = valid & (~known)
    labels[other_occupied] = 6

    return labels

def roi_mask(points_xyz: np.ndarray) -> np.ndarray:
    x = points_xyz[:, 0]
    y = points_xyz[:, 1]
    z = points_xyz[:, 2]
    return (
        (x > X_MIN) & (x < X_MAX) &
        (y > Y_MIN) & (y < Y_MAX) &
        (z > Z_MIN) & (z < Z_MAX)
    )

def split_indices(n: int, val_ratio: float, seed: int):
    idx_all = list(range(n))
    random.Random(seed).shuffle(idx_all)
    n_val = max(1, int(n * val_ratio))
    idx_val = sorted(idx_all[:n_val])
    idx_tr  = sorted(idx_all[n_val:])
    return idx_tr, idx_val

def bincount_final(labels_final: np.ndarray) -> np.ndarray:
    """labels_final dans {0..6} ; retourne hist (7,)"""
    if labels_final.size == 0:
        return np.zeros((NUM_CLASSES,), dtype=np.int64)
    bc = np.bincount(labels_final.astype(np.int64), minlength=NUM_CLASSES)
    return bc[:NUM_CLASSES].astype(np.int64)

def bincount_raw(labs_raw: np.ndarray) -> np.ndarray:
    """
    labs_raw dans {-2,-1,1..22} (+ éventuellement 255).
    Retourne hist RAW dans l'ordre RAW_CLASS_IDS.
    """
    if labs_raw.size == 0:
        return np.zeros((RAW_NUM_CLASSES,), dtype=np.int64)

    labs_raw = labs_raw.astype(np.int16, copy=False)
    hist = np.zeros((RAW_NUM_CLASSES,), dtype=np.int64)

    # map id -> index (petit dict, très stable)
    id_to_idx = {cid: i for i, cid in enumerate(RAW_CLASS_IDS)}

    u, c = np.unique(labs_raw, return_counts=True)
    for ui, ci in zip(u.tolist(), c.tolist()):
        if ui in id_to_idx:
            hist[id_to_idx[ui]] += int(ci)
        # ignore tout ce qui n'est pas dans {-2,-1,1..22} (ex: 255)
    return hist

def scan_root(root_dir: str, val_ratio: float = 0.15, seed: int = 42, max_frames: int | None = None):
    """
    root_dir: dossier qui contient des sous-dossiers, chacun avec points/frame_*.npz
    """
    root = Path(root_dir)
    subdirs = sorted([d for d in root.iterdir() if d.is_dir()])
    if not subdirs:
        raise FileNotFoundError(f"Pas de sous-dossiers dans {root}")

    results = {}
    for sd in subdirs:
        point_files = sorted((sd / "points").glob("frame_*.npz"))
        if not point_files:
            print(f"[WARN] Aucun .npz dans {sd / 'points'} — skip")
            continue

        if max_frames is not None:
            point_files = point_files[:max_frames]

        idx_tr, idx_va = split_indices(len(point_files), val_ratio, seed)

        splits = {
            "train": [point_files[i] for i in idx_tr],
            "val":   [point_files[i] for i in idx_va],
        }

        results[sd.name] = {}
        for split_name, files in splits.items():
            hist_final = np.zeros((NUM_CLASSES,), dtype=np.int64)
            hist_raw_roi = np.zeros((RAW_NUM_CLASSES,), dtype=np.int64)
            npts_after_roi = []

            # stats raw (debug global)
            raw_unique = {}
            n_raw_total = 0

            for fp in files:
                data = np.load(fp, mmap_mode="r")
                pts = data["points"].astype(np.float32, copy=False)
                labs_raw = (
                    data["labels"].astype(np.int16, copy=False)
                    if "labels" in data
                    else np.full((pts.shape[0],), -1, dtype=np.int16)
                )

                # RAW stats (avant ROI)
                u0, c0 = np.unique(labs_raw, return_counts=True)
                for ui, ci in zip(u0.tolist(), c0.tolist()):
                    raw_unique[ui] = raw_unique.get(ui, 0) + int(ci)
                n_raw_total += int(labs_raw.shape[0])

                # ROI
                m = roi_mask(pts[:, :3])
                labs_raw_roi = labs_raw[m]

                # Histogramme RAW (post-ROI) => toutes les classes NPZ
                hist_raw_roi += bincount_raw(labs_raw_roi)

                # Histogramme FINAL (post-ROI) => classes training 0..6
                labels_final = map_labels_like_training(labs_raw_roi)
                valid = (labels_final != IGNORE_LABEL)
                labels_final = labels_final[valid]

                npts_after_roi.append(int(labels_final.shape[0]))
                hist_final += bincount_final(labels_final)

            results[sd.name][split_name] = {
                "hist_final": hist_final,
                "hist_raw_roi": hist_raw_roi,
                "npts_after_roi": np.array(npts_after_roi, dtype=np.int64),
                "n_frames": len(files),
                "raw_unique": raw_unique,
                "n_raw_total": n_raw_total,
            }

    return results

def aggregate_results(results: dict, split_name: str):
    hist_final = np.zeros((NUM_CLASSES,), dtype=np.int64)
    hist_raw_roi = np.zeros((RAW_NUM_CLASSES,), dtype=np.int64)
    npts = []
    n_frames = 0
    for _, ds_data in results.items():
        if split_name not in ds_data:
            continue
        hist_final += ds_data[split_name]["hist_final"]
        hist_raw_roi += ds_data[split_name]["hist_raw_roi"]
        npts.append(ds_data[split_name]["npts_after_roi"])
        n_frames += ds_data[split_name]["n_frames"]
    npts = np.concatenate(npts, axis=0) if npts else np.array([], dtype=np.int64)
    return hist_final, hist_raw_roi, npts, n_frames

def save_barplot(hist: np.ndarray, out_png: str, title: str, names: list[str]):
    total = int(hist.sum())
    pct = (hist / max(total, 1)) * 100.0

    plt.figure(figsize=(max(10, 0.35 * len(hist)), 4))
    x = np.arange(len(hist))
    plt.bar(x, hist)
    plt.xticks(x, names, rotation=60, ha="right")
    plt.ylabel("Count")
    plt.title(f"{title} | total={total:,}")
    for i in range(len(hist)):
        if hist[i] > 0:
            plt.text(i, hist[i], f"{pct[i]:.1f}%", ha="center", va="bottom", fontsize=8)
    plt.tight_layout()
    os.makedirs(Path(out_png).parent, exist_ok=True)
    plt.savefig(out_png, dpi=150)
    plt.close()

def pretty_print_final(hist: np.ndarray, npts: np.ndarray, name: str):
    total = int(hist.sum())
    print(f"\n=== {name} (FINAL 0..6) ===")
    print(f"Total labels counted (post-ROI, ignore 255): {total:,}")
    for i in range(NUM_CLASSES):
        p = (hist[i] / max(total, 1)) * 100.0
        print(f"{CLASS_NAMES[i]:>14s}: {hist[i]:>12,}   ({p:6.2f}%)")
    if npts.size:
        print(f"#points after ROI per frame: min={int(npts.min()):,}  mean={float(npts.mean()):,.1f}  max={int(npts.max()):,}")
    else:
        print("No frames / no points found.")

def pretty_print_raw(hist_raw: np.ndarray, name: str):
    total = int(hist_raw.sum())
    print(f"\n=== {name} (RAW NPZ -2/-1/1..22, post-ROI) ===")
    print(f"Total raw labels counted (post-ROI): {total:,}")
    for cid, cname, count in zip(RAW_CLASS_IDS, RAW_CLASS_NAMES, hist_raw.tolist()):
        p = (count / max(total, 1)) * 100.0
        print(f"{cname:>16s}: {count:>12,}   ({p:6.2f}%)")

def print_raw_summary(raw_unique: dict, n_raw_total: int, title: str, topk: int = 30):
    print(f"\n--- RAW LABELS SUMMARY (avant ROI): {title} ---")
    print(f"Total raw points scanned: {n_raw_total:,}")
    items = sorted(raw_unique.items(), key=lambda kv: kv[1], reverse=True)
    for k, v in items[:topk]:
        p = (v / max(n_raw_total, 1)) * 100.0
        print(f"raw {str(k):>4s}: {v:>12,}   ({p:6.2f}%)")
    if len(items) > topk:
        print(f"... ({len(items)-topk} autres valeurs raw)")

def main():
    # >>>>>>> A MODIFIER <<<<<<<<
    DATA_ROOT = r"/content/DataSet/finalDataset/finalDataset"
    OUT_DIR   = r"/content/drive/MyDrive/OccMLP/dataset_stats"
    VAL_RATIO = 0.15
    SEED      = 42
    MAX_FRAMES_PER_SUBDIR = 500
    # --------------------------

    results = scan_root(DATA_ROOT, val_ratio=VAL_RATIO, seed=SEED, max_frames=MAX_FRAMES_PER_SUBDIR)

    # agrégation globale
    hist_tr_final, hist_tr_raw, npts_tr, nframes_tr = aggregate_results(results, "train")
    hist_va_final, hist_va_raw, npts_va, nframes_va = aggregate_results(results, "val")

    print(f"Scanned subdirs: {len(results)}")
    print(f"Train frames: {nframes_tr} | Val frames: {nframes_va}")

    # prints
    pretty_print_raw(hist_tr_raw, "GLOBAL TRAIN")
    pretty_print_final(hist_tr_final, npts_tr, "GLOBAL TRAIN")

    pretty_print_raw(hist_va_raw, "GLOBAL VAL")
    pretty_print_final(hist_va_final, npts_va, "GLOBAL VAL")

    # plots
    save_barplot(hist_tr_raw, str(Path(OUT_DIR) / "hist_train_raw_npz.png"),
                 "Dataset histogram - TRAIN (RAW NPZ -2/-1/1..22, post-ROI)", RAW_CLASS_NAMES)
    save_barplot(hist_va_raw, str(Path(OUT_DIR) / "hist_val_raw_npz.png"),
                 "Dataset histogram - VAL (RAW NPZ -2/-1/1..22, post-ROI)", RAW_CLASS_NAMES)

    save_barplot(hist_tr_final, str(Path(OUT_DIR) / "hist_train_final.png"),
                 "Dataset histogram - TRAIN (final 0..6)", CLASS_NAMES)
    save_barplot(hist_va_final, str(Path(OUT_DIR) / "hist_val_final.png"),
                 "Dataset histogram - VAL (final 0..6)", CLASS_NAMES)

    # Optionnel : par sous-dossier + raw debug
    for ds_name, ds_data in results.items():
        for split in ("train", "val"):
            h_final = ds_data[split]["hist_final"]
            h_raw = ds_data[split]["hist_raw_roi"]

            save_barplot(h_raw, str(Path(OUT_DIR) / f"hist_{ds_name}_{split}_raw_npz.png"),
                         f"{ds_name} - {split} (RAW NPZ -2/-1/1..22, post-ROI)", RAW_CLASS_NAMES)
            save_barplot(h_final, str(Path(OUT_DIR) / f"hist_{ds_name}_{split}_final.png"),
                         f"{ds_name} - {split} (final 0..6)", CLASS_NAMES)

            print_raw_summary(ds_data[split]["raw_unique"], ds_data[split]["n_raw_total"], f"{ds_name} / {split}")

    print(f"\nSaved plots to: {OUT_DIR}")

if __name__ == "__main__":
    main()


Scanned subdirs: 1
Train frames: 425 | Val frames: 75

=== GLOBAL TRAIN (RAW NPZ -2/-1/1..22, post-ROI) ===
Total raw labels counted (post-ROI): 9,793,215
        -2 Empty:    2,014,270   ( 20.57%)
      -1 Unknown:    1,673,476   ( 17.09%)
      1 Building:    2,771,933   ( 28.30%)
         2 Fence:    1,701,222   ( 17.37%)
         3 Other:      466,174   (  4.76%)
    4 Pedestrian:            0   (  0.00%)
          5 Pole:       56,554   (  0.58%)
      6 RoadLine:       53,991   (  0.55%)
          7 Road:           84   (  0.00%)
      8 SideWalk:        4,744   (  0.05%)
    9 Vegetation:      687,972   (  7.02%)
     10 Vehicles:       43,728   (  0.45%)
         11 Wall:            0   (  0.00%)
  12 TrafficSign:            0   (  0.00%)
          13 Sky:            0   (  0.00%)
       14 Ground:       66,186   (  0.68%)
       15 Bridge:       15,868   (  0.16%)
    16 RailTrack:        4,017   (  0.04%)
    17 GuardRail:            0   (  0.00%)
 18 TrafficLight:        2,1

In [ ]:
!pip -q install plotly

import numpy as np
import plotly.graph_objects as go
import plotly.express as px

NPZ_PATH = "/content/DataSet/finalDataset/finalDataset/100k/points/frame_000000.npz"
MAX_POINTS = 60000  # ajuste si besoin

data = np.load(NPZ_PATH, allow_pickle=True)
pts = data["points"].astype(np.float32)
labels = data["labels"].astype(np.int16)

# Optionnel (si présent dans ton npz)
class_ids = data["class_ids"].astype(np.int16) if "class_ids" in data else None
class_names = data["class_names"] if "class_names" in data else None

print("Points:", pts.shape)
u = np.unique(labels)
print("Unique labels:", u)

# Downsample
if len(pts) > MAX_POINTS:
    idx = np.random.choice(len(pts), MAX_POINTS, replace=False)
    pts = pts[idx]
    labels = labels[idx]

# Build name per label (fallback si class_names pas dans le npz)
id_to_name = {}
if class_ids is not None and class_names is not None:
    # class_names peut être dtype=object -> convertir proprement
    cn = [str(x) for x in class_names.tolist()]
    for cid, name in zip(class_ids.tolist(), cn):
        id_to_name[int(cid)] = name
else:
    # fallback minimal
    id_to_name[-2] = "Empty"
    id_to_name[-1] = "Unknown"
    for k in range(1, 40):  # large, au cas où (ex: 24)
        id_to_name[k] = f"Class {k}"

# Couleurs : palette qualitative + fallback cyclique
palette = px.colors.qualitative.Safe + px.colors.qualitative.Set3 + px.colors.qualitative.Bold
uniq = np.unique(labels)

# Map label -> color
label_to_color = {int(l): palette[i % len(palette)] for i, l in enumerate(uniq)}

# Tracer une trace par classe (permet légende + couleur fixe)
fig = go.Figure()

for l in uniq:
    l = int(l)
    mask = (labels == l)
    if mask.sum() == 0:
        continue

    name = id_to_name.get(l, f"Class {l}")
    fig.add_trace(go.Scatter3d(
        x=pts[mask, 0],
        y=pts[mask, 1],
        z=pts[mask, 2],
        mode="markers",
        name=f"{l}: {name} ({mask.sum()})",
        marker=dict(size=2, color=label_to_color[l], opacity=0.9),
    ))

fig.update_layout(
    title=f"NPZ 3D view (classes auto) — {NPZ_PATH.split('/')[-1]}",
    scene=dict(
        xaxis_title="X", yaxis_title="Y", zaxis_title="Z",
        xaxis=dict(range=[-16, 16]),
        yaxis=dict(range=[-16, 16]),
        zaxis=dict(range=[-2, 8]),
        aspectmode="cube",
    ),
    legend=dict(itemsizing="constant"),
    height=750,
)

fig.show()


Points: (37324, 3)
Unique labels: [-2 -1  1  2  3  6  8  9 10 20 22 24]


In [ ]:
pip install -U nvidia-modelopt[all]

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 79.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.5/89.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.2/283.2 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.2/234.2 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.8/372.8 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 126.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
SurroundOcc Optimisé TensorRT (Jetson Thor/Orin)
- Full Pipeline (BiFPN, Deconv, Viz)
- Attention Fusion (Method 2: Primitive Ops)
- GatherND (Mode ND 5D Zero-Copy)
- QAT/LSS DESACTIVE -> Linear standard uniquement
"""

import os
import math
import random
import argparse
from pathlib import Path
from typing import List, Tuple

import numpy as np
from PIL import Image, ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import timm
import matplotlib
import time
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

from torch.backends.cuda import sdp_kernel
import torch._inductor.config as inductor_config  # noqa: F401

# =========================================================
# MODELOPT / QAT DESACTIVE
# =========================================================
HAS_MODELOPT = False
qnn = nn
print("ℹ️ QAT désactivé : couches nn standard.")

# =========================================================
# OPTIMISATIONS GPU / TF32
# =========================================================
if hasattr(torch.backends.cuda.matmul, "allow_tf32"):
    torch.backends.cuda.matmul.allow_tf32 = True
if hasattr(torch.backends.cuda.matmul, "fp32_precision"):
    torch.backends.cuda.matmul.fp32_precision = "high"

torch.backends.cudnn.benchmark = True
if hasattr(torch.backends.cudnn, "allow_tf32"):
    torch.backends.cudnn.allow_tf32 = True
if hasattr(torch.backends.cudnn, "conv") and hasattr(torch.backends.cudnn.conv, "fp32_precision"):
    torch.backends.cudnn.conv.fp32_precision = "tf32"

torch.set_float32_matmul_precision("high")

# =========================================================
# CONFIG GLOBALE
# =========================================================a
CAM_NAMES = ["front", "back", "left", "right", "front_left", "front_right", "back_left", "back_right"]



NUM_CLASSES = 7
EMPTY_LABEL = 0
UNKNOWN_LABEL = 1
ROAD_LABEL = 2
SIDEWALK_LABEL = 3
VEGETATION_LABEL = 4
VEHICULES_LABEL = 5
OCCUPIED_LABEL = 6

IMAGE_SIZE = (512, 384)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

DEFAULT_X_RANGE_M = [-16.0, 16.0]
DEFAULT_Y_RANGE_M = [-16.0, 16.0]
DEFAULT_Z_RANGE_M = [-8.0, 8.0]
N_POINTS_FIXED = 131_072

CUBE_SIZE_M = 0.1
MAX_POINTS_VIZ = 75_000

OCC_COLORS = [
    (1.0, 1.0, 1.0),  # 0 Empty
    (0.6, 0.6, 0.6),  # 1 Unknown
    (0.0, 0.0, 1.0),  # 2 Road blue
    (1.0, 1.0, 0.0),  # 3 SideWalk yellow
    (0.0, 1.0, 0.0),  # 4 Vegetation green
    (1.0, 0.0, 0.0),  # 5 Vehicules red
    (1.0, 0.0, 1.0),  # 6 Occupied Magenta violet
]

# =========================================================
# UTILS
# =========================================================
def set_seed(s: int = 42):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)

def save_checkpoint(path: str, model, optimizer, epoch: int, best_acc: float):
    ckpt = {
        "model": model.state_dict(),
        "optimizer": optimizer.state_dict() if optimizer is not None else None,
        "epoch": epoch,
        "best_acc": best_acc,
    }
    torch.save(ckpt, path)

def load_checkpoint(path: str, model, optimizer=None, map_location="cpu"):
    ckpt = torch.load(path, map_location=map_location)
    model.load_state_dict(ckpt["model"])
    start_epoch = ckpt.get("epoch", 0)
    best_acc = ckpt.get("best_acc", 0.0)
    if optimizer is not None and ckpt.get("optimizer", None) is not None:
        optimizer.load_state_dict(ckpt["optimizer"])
    return start_epoch, best_acc

def enable_flash_attention():
    if not torch.cuda.is_available():
        return
    try:
        torch.backends.cuda.enable_flash_sdp(True)
        torch.backends.cuda.enable_mem_efficient_sdp(True)
        torch.backends.cuda.enable_math_sdp(False)
    except AttributeError:
        sdp_kernel(enable_flash=True, enable_mem_efficient=True, enable_math=False)


def _fmt_time(seconds: float) -> str:
    seconds = max(0.0, float(seconds))
    m, s = divmod(int(seconds + 0.5), 60)
    h, m = divmod(m, 60)
    if h > 0:
        return f"{h:02d}:{m:02d}:{s:02d}"
    return f"{m:02d}:{s:02d}"

class IterMeter:
    """
    Mesure it/s + ETA avec une moyenne glissante (stable, lisible).
    """
    def __init__(self, total_iters: int, window: int = 10):
        self.total = int(total_iters)
        self.window = int(window)
        self.t0 = time.perf_counter()
        self.last_t = self.t0
        self.n = 0
        self.hist = []  # durées it
        self.last_report = self.t0

    def update(self, it_incr: int = 1) -> None:
        now = time.perf_counter()
        dt = now - self.last_t
        self.last_t = now
        self.n += it_incr
        self.hist.append(dt)
        if len(self.hist) > self.window:
            self.hist.pop(0)

    def stats(self) -> tuple[float, float]:
        # returns (it_per_s, eta_s)
        if not self.hist:
            return 0.0, float("inf")
        avg_dt = sum(self.hist) / len(self.hist)
        it_per_s = 1.0 / max(avg_dt, 1e-12)
        remaining = max(self.total - self.n, 0)
        eta_s = remaining * avg_dt
        return it_per_s, eta_s



# =========================================================
# VISU 3D
# =========================================================
def _cube_vertices(center, size):
    x, y, z = center
    s = size / 2.0
    return np.array([
        [x - s, y - s, z - s], [x + s, y - s, z - s], [x + s, y + s, z - s], [x - s, y + s, z - s],
        [x - s, y - s, z + s], [x + s, y - s, z + s], [x + s, y + s, z + s], [x - s, y + s, z + s],
    ])

def _cube_faces(verts):
    return [
        [verts[0], verts[1], verts[2], verts[3]], [verts[4], verts[5], verts[6], verts[7]],
        [verts[0], verts[1], verts[5], verts[4]], [verts[2], verts[3], verts[7], verts[6]],
        [verts[1], verts[2], verts[6], verts[5]], [verts[4], verts[7], verts[3], verts[0]],
    ]

def save_3d_cubes(points_m: np.ndarray, labels: np.ndarray, out_path: str, cube_size: float = CUBE_SIZE_M):
    N = points_m.shape[0]
    if N == 0: return

    # --- OPTIMISATION 1 : Downsampling pour la viz ---
    # Même sur A100, afficher 130k cubes est illisible. 30k-40k suffisent largement.
    max_viz = 70_000
    if N > max_viz:
        idx = np.random.choice(N, size=max_viz, replace=False)
        pts = points_m[idx]
        labs = labels[idx]
    else:
        pts, labs = points_m, labels

    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection="3d")

    all_faces = []
    face_colors = []

    s = cube_size / 2.0

    # --- OPTIMISATION 2 : Vectorisation manuelle des faces ---
    for i in range(len(pts)):
        x, y, z = pts[i]
        c_int = int(labs[i])
        if c_int == 255: continue

        color = OCC_COLORS[c_int] if c_int < len(OCC_COLORS) else (0.5, 0.5, 0.5)

        # Sommets du cube
        v = np.array([
            [x-s,y-s,z-s], [x+s,y-s,z-s], [x+s,y+s,z-s], [x-s,y+s,z-s],
            [x-s,y-s,z+s], [x+s,y-s,z+s], [x+s,y+s,z+s], [x-s,y+s,z+s]
        ])

        # Les 6 faces du cube
        cube_faces = [
            [v[0],v[1],v[2],v[3]], [v[4],v[5],v[6],v[7]], # bas, haut
            [v[0],v[1],v[5],v[4]], [v[2],v[3],v[7],v[6]], # cotés
            [v[1],v[2],v[6],v[5]], [v[4],v[7],v[3],v[0]]  # cotés
        ]

        all_faces.extend(cube_faces)
        # On ajoute la couleur pour les 6 faces
        face_colors.extend([color] * 6)

    # --- OPTIMISATION 3 : Un seul appel à Poly3DCollection ---
    poly3d = Poly3DCollection(all_faces, facecolors=face_colors, edgecolors=None, linewidths=0)
    ax.add_collection3d(poly3d)

    # Réglage des limites (fixe pour éviter que l'OS calcule tout seul)
    ax.set_xlim(-16, 16)
    ax.set_ylim(-16, 16)
    ax.set_zlim(-2, 8)
    ax.view_init(elev=20, azim=45)

    plt.tight_layout()
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    fig.savefig(out_path, dpi=100)
    plt.close(fig)
# =========================================================
# CAM UTILS
# =========================================================
def deg2rad(x):
    return x * math.pi / 180.0

def rot_mat_yaw_pitch(yaw_deg: float, pitch_deg: float) -> torch.Tensor:
    yaw = deg2rad(yaw_deg)
    pitch = deg2rad(pitch_deg)
    cy, sy = math.cos(yaw), math.sin(yaw)
    cp, sp = math.cos(pitch), math.sin(pitch)
    Rz = torch.tensor([[cy, -sy, 0.0], [sy, cy, 0.0], [0.0, 0.0, 1.0]], dtype=torch.float32)
    Ry = torch.tensor([[cp, 0.0, sp], [0.0, 1.0, 0.0], [-sp, 0.0, cp]], dtype=torch.float32)
    return Rz @ Ry

def intrinsics_from_fov(fov_deg: float, img_w: int, img_h: int) -> torch.Tensor:
    f = 0.5 * img_w / math.tan(0.5 * deg2rad(fov_deg))
    cx, cy = img_w * 0.5, img_h * 0.5
    K = torch.tensor([[f, 0.0, cx], [0.0, f, cy], [0.0, 0.0, 1.0]], dtype=torch.float32)
    return K

# =========================================================
# DATASET
# =========================================================
class ImplicitOccDataset(Dataset):
    def __init__(
        self,
        root: str,
        split: str = "train",
        val_ratio: float = 0.15,
        seed: int = 42,
        expect_hw: tuple = (384, 512),
        normalize: bool = True,
        empty_ratio: float = 0.0,
        # Ajout des paramètres de limitation de points
        min_points: int = 45_000,
        max_points: int = 60_000,
    ):
        self.root = Path(root)
        self.expect_hw = expect_hw
        self.normalize = normalize
        self.empty_ratio = empty_ratio
        self.min_points = min_points
        self.max_points = max_points

        point_files = sorted((self.root / "points").glob("frame_*.npz"))
        if not point_files:
            raise FileNotFoundError(f"Aucun .npz dans {self.root / 'points'}")

        random.seed(seed)
        idx_all = list(range(len(point_files)))
        random.shuffle(idx_all)
        n_val = max(1, int(len(idx_all) * val_ratio))
        keep = sorted(idx_all[n_val:]) if split == "train" else sorted(idx_all[:n_val])
        self.point_files = [point_files[i] for i in keep]

        print(f"[{split}] frames: {len(self.point_files)} | root={self.root}")
        print(f"[{split}] Subsampling: target between {min_points} and {max_points} pts.")

        self.mean = torch.tensor([0.485, 0.456, 0.406]).view(1, 1, 3)
        self.std = torch.tensor([0.229, 0.224, 0.225]).view(1, 1, 3)

    def __len__(self):
        return len(self.point_files)

    def _load_images(self, stem: str) -> torch.Tensor:
        TARGET_H, TARGET_W = 384, 512
        imgs = []
        for name in CAM_NAMES:
            p = self.root / "images" / f"{stem}_{name}.jpg"
            if not p.exists(): p = p.with_suffix(".png")
            if not p.exists(): raise FileNotFoundError(f"Image manquante: {p}")

            # im = Image.open(p).convert("RGB")
            im =  Image.open(p)
            orig_w, orig_h = im.size
            if orig_h != 384 or orig_w != 512:
                assert print("failed, with size:", orig_w, orig_h)

            arr = np.asarray(im, dtype=np.float32) / 255.0
            t = torch.from_numpy(arr)
            if self.normalize:
                t = (t - self.mean) / self.std
            t = t.permute(2, 0, 1).contiguous()
            imgs.append(t)
        return torch.stack(imgs, dim=0).contiguous()

    def __getitem__(self, i: int):
        point_path = self.point_files[i]
        stem = point_path.stem
        data = np.load(point_path, mmap_mode="r")
        pts = data["points"].astype(np.float32)
        labs_raw = data["labels"].astype(np.int16) if "labels" in data else np.full((pts.shape[0],), 1, dtype=np.int16)

        # Mappage des labels (ton code original)
        # =====================================================
        # Mappage labels cohérent avec le .npz (implicit occupancy)
        # labs_raw ∈ { -2 (Empty), -1 (Unknown), 1..22 (CARLA) }
        # On garde: Unknown, Empty, Road, Sidewalk, Vegetation, Vehicles
        # Le reste => Occupied
        # =====================================================
        labels = np.full_like(labs_raw, fill_value=255, dtype=np.int16)

        valid = (labs_raw != 255)  # sécurité (normalement inutile avec ton .npz)
        empty_mask   = valid & (labs_raw == -2)
        unknown_mask = valid & (labs_raw == -1)

        road_mask      = valid & (labs_raw == 7)   # CARLA Road
        sidewalk_mask  = valid & (labs_raw == 8)   # CARLA SideWalk
        veget_mask     = valid & (labs_raw == 9)   # CARLA Vegetation
        vehicle_mask   = valid & (labs_raw == 10)  # CARLA Vehicles

        labels[empty_mask] = EMPTY_LABEL
        labels[unknown_mask] = UNKNOWN_LABEL
        labels[road_mask] = ROAD_LABEL
        labels[sidewalk_mask] = SIDEWALK_LABEL
        labels[veget_mask] = VEGETATION_LABEL
        labels[vehicle_mask] = VEHICULES_LABEL

        # Tout le reste (CARLA 1..22 hors 7/8/9/10) => OCCUPIED
        known_masks = empty_mask | unknown_mask | road_mask | sidewalk_mask | veget_mask | vehicle_mask
        other_occupied_mask = valid & (~known_masks)
        labels[other_occupied_mask] = OCCUPIED_LABEL

        xyz_m = torch.from_numpy(pts[:, :3])
        occ_labels = torch.from_numpy(labels.astype(np.int16))

        # 1. Filtrage ROI
        eps = 5e-6
        X_MIN, X_MAX = -15 + eps, 15 - eps
        Y_MIN, Y_MAX = -15 + eps, 15 - eps
        Z_MIN, Z_MAX = -2 + eps, 6 - eps

        in_roi = (xyz_m[:, 0] > X_MIN) & (xyz_m[:, 0] < X_MAX) & \
                 (xyz_m[:, 1] > Y_MIN) & (xyz_m[:, 1] < Y_MAX) & \
                 (xyz_m[:, 2] > Z_MIN) & (xyz_m[:, 2] < Z_MAX)

        xyz_m = xyz_m[in_roi]
        occ_labels = occ_labels[in_roi]

        # 2. Sous-échantillonnage aléatoire (Nouveau)
        num_available = xyz_m.shape[0]
        # On définit une cible aléatoire entre min et max pour cette frame précise
        target_n = random.randint(self.min_points, self.max_points)

        if num_available > target_n:
            # On génère des indices aléatoires sans remise
            # Utilisation de torch.randperm pour la rapidité sur de grands ensembles
            indices = torch.randperm(num_available)[:target_n]
            xyz_m = xyz_m[indices]
            occ_labels = occ_labels[indices]
        # Si num_available <= target_n, on garde tout (xyz_m et occ_labels restent inchangés)

        imgs = self._load_images(stem)
        return imgs, xyz_m, occ_labels, {"stem": stem}

def collate_implicit(batch):
    imgs_list, xyz_list, lab_list, meta_list = zip(*batch)
    imgs = torch.stack(imgs_list, dim=0)
    max_N = max(x.shape[0] for x in xyz_list)
    pts_pad, labels_pad, mask = [], [], []
    for xyz, lab in zip(xyz_list, lab_list):
        N = xyz.shape[0]
        pts_pad.append(F.pad(xyz, (0, 0, 0, max_N - N), value=0.0))
        labels_pad.append(F.pad(lab, (0, max_N - N), value=255))
        m = torch.zeros(max_N, dtype=torch.bool)
        m[:N] = True
        mask.append(m)
    return imgs, torch.stack(pts_pad, 0), torch.stack(labels_pad, 0), torch.stack(mask, 0), meta_list

# =========================================================
# BACKBONE + BiFPN
# =========================================================
class BiFPN(nn.Module):
    def __init__(self, in_chs: dict, dim: int = 320):
        super().__init__()

        def conv1x1_bn(in_c, out_c):
            return nn.Sequential(nn.Conv2d(in_c, out_c, 1, bias=False), nn.BatchNorm2d(out_c))

        def sep_conv(ch):
            return nn.Sequential(
                nn.Conv2d(ch, ch, 3, padding=1, bias=False),
                nn.BatchNorm2d(ch),
            )

        def down_bn(ch):
            return nn.Sequential(nn.Conv2d(ch, ch, 3, stride=2, padding=1, bias=False), nn.BatchNorm2d(ch))

        def up_bn(ch):
            return nn.Sequential(nn.ConvTranspose2d(ch, ch, kernel_size=2, stride=2, bias=False))

        self.lat_p2 = conv1x1_bn(in_chs["p2"], dim)
        self.lat_p3 = conv1x1_bn(in_chs["p3"], dim)
        self.lat_p4 = conv1x1_bn(in_chs["p4"], dim)
        self.up_p4 = up_bn(dim)
        self.up_p3 = up_bn(dim)
        self.down_p2 = down_bn(dim)
        self.down_p3 = down_bn(dim)
        self.td_p3 = sep_conv(dim)
        self.td_p2 = sep_conv(dim)
        self.bu_p3 = sep_conv(dim)
        self.bu_p4 = sep_conv(dim)
        self.act = nn.ReLU(inplace=True)

    def forward(self, feats: dict):
        p2 = self.lat_p2(feats["p2"])
        p3 = self.lat_p3(feats["p3"])
        p4 = self.lat_p4(feats["p4"])
        # Top-down
        p3_td = self.td_p3(self.act(p3 + self.up_p4(p4)))
        p2_td = self.td_p2(self.act(p2 + self.up_p3(p3_td)))
        # Bottom-up
        p3_out = self.bu_p3(self.act(p3_td + self.down_p2(p2_td)))
        p4_out = self.bu_p4(self.act(p4 + self.down_p3(p3_out)))
        return {"p2": p2_td, "p3": p3_out, "p4": p4_out}

# =========================================================
# ATTENTION (LINEAR STANDARD)
# =========================================================
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class TRTFusedAttention(nn.Module):
    """
    Attention "primitive ops" compatible export TRT:
    (B, Sq, C) + (B, Skv, C) -> (B, Sq, C)
    """
    def __init__(self, dim, heads=8):
        super().__init__()
        assert dim % heads == 0
        self.dim = dim
        self.heads = heads
        self.head_dim = dim // heads
        self.scale = 1.0 / math.sqrt(self.head_dim)
        self.q_proj = nn.Linear(dim, dim, bias=False)
        self.k_proj = nn.Linear(dim, dim, bias=False)
        self.v_proj = nn.Linear(dim, dim, bias=False)
        self.out_proj = nn.Linear(dim, dim, bias=False)

    def forward(self, q: torch.Tensor, kv: torch.Tensor) -> torch.Tensor:
        """
        q: (B, Sq, C)
        kv: (B, Skv, C)
        """
        B, Sq, C = q.shape
        Skv = kv.shape[1]
        Q = self.q_proj(q).reshape(B, Sq, self.heads, self.head_dim).permute(0, 2, 1, 3)
        K = self.k_proj(kv).reshape(B, Skv, self.heads, self.head_dim).permute(0, 2, 1, 3)
        V = self.v_proj(kv).reshape(B, Skv, self.heads, self.head_dim).permute(0, 2, 1, 3)
        out = F.scaled_dot_product_attention(Q, K, V, scale=self.scale)
        out = out.permute(0, 2, 1, 3).reshape(B, Sq, C)
        return self.out_proj(out)

class PositionalTransformer3D(nn.Module):
    def __init__(self, cams=8, query_voxel=(16, 16, 4), spatial_dim = 512, spatial_heads=8, spatial_layers=8, drop=0.0,
                num_experts=0, capacity_factor=0, pe_dtype=torch.float16):
        super().__init__()
        self.dim = spatial_dim
        self.lvl_order = ["p4", "p3"]                 # mêmes niveaux que tu utilises
        self.level_embed = nn.Embedding(len(self.lvl_order), self.dim)
        nn.init.trunc_normal_(self.level_embed.weight, std=0.02)
        self.cams = cams
        self.query_voxel = tuple(query_voxel)
        self.dim_BiFPN=128
        self.input_projs = nn.ModuleDict({ # total tocken : 2520
            "p4": nn.Sequential(
                nn.AvgPool2d(kernel_size=2, stride=2),
            ),
            "p3": nn.Sequential(
                nn.AvgPool2d(kernel_size=4, stride=4),
            ),
            "p2": nn.Sequential(
                nn.AvgPool2d(kernel_size=16, stride=16),
            ),
        })
        self.input_to_dim = nn.Sequential(
            nn.Linear(self.dim_BiFPN, self.dim, bias=False),
            nn.LayerNorm(self.dim),
            # nn.Linear(self.dim, dim, bias=False),
        )
        self.cam_embed = nn.Embedding(cams, self.dim)
        self.register_buffer("cam_ids", torch.arange(cams, dtype=torch.long), persistent=False)
        n_q = query_voxel[0] * query_voxel[1] * query_voxel[2]
        self.query = nn.Parameter(torch.randn(1, n_q, self.dim) * 0.02)
        nn.init.trunc_normal_(self.query, std=0.02, a=-0.04, b=0.04)

        # --- OPTIMISATION 1 : PRE-CALCUL PE ---
        # On construit le PE une seule fois ici au lieu de le faire dans le forward
        pe = self._build_pe_3d(*query_voxel, self.dim, torch.float32) # On calcul en FP32 pour la précision des sin/cos
        self.register_buffer("pe_fixed", pe, persistent=False)
        # --------------------------------------
        self.layers = nn.ModuleList([
            nn.ModuleDict({
                "cross": TRTFusedAttention(dim=self.dim, heads=spatial_heads),
                "ln1": nn.LayerNorm(self.dim),
                "self":  TRTFusedAttention(dim=self.dim, heads=spatial_heads),
                "ln2": nn.LayerNorm(self.dim),
                "ffn":   nn.Sequential(
                    nn.Linear(self.dim, self.dim * 4),
                    nn.GELU(),
                    nn.Dropout(p=0.1),
                    nn.Linear(self.dim * 4, self.dim),
                ),
            }) for _ in range(spatial_layers)
        ])



    def _posenc_2d_sincos(self, h: int, w: int, dim: int, device, dtype):
        """
        Positional Encoding 2D sin/cos: (h*w, dim)
        dim doit être multiple de 4.
        """
        assert dim % 4 == 0, "dim doit être multiple de 4 pour un sincos 2D"

        yy, xx = torch.meshgrid(
            torch.arange(h, device=device),
            torch.arange(w, device=device),
            indexing="ij",
        )
        yy = yy.reshape(-1).to(dtype)
        xx = xx.reshape(-1).to(dtype)

        # denom tensor-only (évite max Python)
        denom_y = torch.tensor(h - 1, device=device, dtype=dtype).clamp_min(1.0)
        denom_x = torch.tensor(w - 1, device=device, dtype=dtype).clamp_min(1.0)

        yy = yy / denom_y
        xx = xx / denom_x

        d = dim // 4
        omega = torch.arange(d, device=device, dtype=dtype) / d
        omega = 1.0 / (10000 ** omega)  # (d,)

        out_x = xx[:, None] * omega[None, :]  # (hw, d)
        out_y = yy[:, None] * omega[None, :]  # (hw, d)

        pe = torch.cat(
            [torch.sin(out_x), torch.cos(out_x), torch.sin(out_y), torch.cos(out_y)],
            dim=1,
        )  # (hw, dim)

        return pe

    def _build_pe_3d(self, qx, qy, qz, dim, dtype):
        """
        Génération de Fourier Random Features (Gaussian Mapping).
        Utilise toute la dimension 'dim' pour un encodage spatial haute-fidélité.
        """
        # 1. Création de la grille de coordonnées [0, 1]
        gx, gy, gz = torch.meshgrid(
            torch.linspace(0, 1, qx),
            torch.linspace(0, 1, qy),
            torch.linspace(0, 1, qz),
            indexing="ij",
        )
        coords = torch.stack([gx, gy, gz], dim=-1).view(-1, 3).to(dtype) # (N, 3)

        # 2. Génération de la matrice de fréquences (Gaussian Matrix)
        # On utilise une seed fixe pour que le PE soit le même à chaque export/run
        # On veut projeter 3 coord vers dim // 2 (pour avoir sin + cos = dim)
        num_fourier_feats = dim // 2
        sigma = 8
        g = torch.Generator()
        g.manual_seed(42)
        b_matrix = torch.randn(3, num_fourier_feats, generator=g, dtype=dtype) * sigma


        # 3. Projection de Fourier : Coords @ B
        # (N, 3) @ (3, dim//2) -> (N, dim//2)
        v_proj = 2.0 * math.pi * (coords @ b_matrix)

        # 4. Sin/Cos et concaténation
        # Résultat : (N, dim)
        pe = torch.cat([torch.sin(v_proj), torch.cos(v_proj)], dim=-1)

        # 5. Reshape final pour le buffer (1, N, dim)
        pe = pe.unsqueeze(0)

        print(f"✅ PE Fourier généré : {pe.shape} (Sigma={sigma})")
        return pe

    def _build_tokens(self, per_level_by_cam):
        lvl_order = self.lvl_order  # ["p4","p3"]
        B, Cams = per_level_by_cam["p4"].shape[:2]

        cam_emb = self.cam_embed.weight.view(1, Cams, 1, self.dim)  # (1, Cams, 1, 512)
        toks_all = []

        for lvl_id, lvl in enumerate(lvl_order):
            feats = per_level_by_cam[lvl]          # (B, Cams, 128, H, W)
            x = feats.flatten(0, 1)                # (B*Cams, 128, H, W)
            x = self.input_projs[lvl](x)           # (B*Cams, 128, H, W) to (B*Cams, 512, h/2, w/2)
            x = torch.nn.functional.pixel_unshuffle(x, downscale_factor=2)
            h, w = x.shape[-2], x.shape[-1]

            # --- Tokenize ---
            x = x.flatten(2).transpose(1, 2)       # (B*Cams, hw, 512)
            # x = self.input_to_dim(x)  plus besoin             # (B*Cams, hw, 512)

            # ✅ 2D positional embedding pour KV (repère x,y)
            pe2d = self._posenc_2d_sincos(h, w, self.dim, x.device, x.dtype)  # (hw, 512)
            x = x + pe2d.unsqueeze(0)              # (B*Cams, hw, 512)

            # Reshape (B, Cams, hw, 512)
            x = x.view(B, Cams, -1, self.dim)

            # ✅ camera embedding
            x = x + cam_emb

            # ✅ level embedding (p3 vs p4)
            x = x + self.level_embed.weight[lvl_id].view(1, 1, 1, self.dim)

            # Flatten cams + spatial
            x = x.flatten(1, 2)                    # (B, Cams*hw, 512)
            toks_all.append(x)

        return torch.cat(toks_all, dim=1)

    def forward(self, per_level_by_cam):
        kv = self._build_tokens(per_level_by_cam)
        B = kv.shape[0]
        # Utilisation du buffer pré-calculé (cast dynamique pour AMP/FP16)
        pe = self.pe_fixed.expand(B, -1, -1).to(kv.dtype)
        q = self.query.expand(B, -1, -1).to(kv.dtype)
        x = q + pe

        for layer in self.layers:
            x = x + layer["cross"](layer["ln1"](x), kv)
            h = layer["ln2"](x)
            x = x + layer["self"](h, h)
            x = x + layer["ffn"](x)

        qx, qy, qz = self.query_voxel
        # Output Channel First pour compatibilité Deconv (B, qx, qy, qz, C) -> puis reshape plus tard
        x = x.reshape(B, qz, qy, qx, self.dim)
        return x

# =========================================================
# DECONV 3D
# =========================================================
class ChannelLastUpsample3d(nn.Module):
    """
    Upsampling 3D (channel-last) : Linear projection -> VoxelShuffle (réarrangement mémoire)
    Input :  (B, D, H, W, Cin)
    Output:  (B, D*s, H*s, W*s, Cout)
    """
    def __init__(self, in_c: int, out_c: int, scale: int = 2):
        super().__init__()
        self.out_c = out_c
        self.scale = int(scale)
        self.expansion = self.scale ** 3  # 2^3 = 8 pour scale=2
        self.proj = nn.Sequential(
            nn.Linear(in_c, out_c * 8, bias=False),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: (B, D, H, W, Cin)
        B, D, H, W, _ = x.shape
        s = self.scale
        Cout = self.out_c

        x = self.proj(x)  # (B, D, H, W, Cout*s^3)

        # Split expansion into (sd, sh, sw, Cout)
        x = x.reshape(B, D, H, W, s, s, s, Cout)  # (B, D, H, W, sd, sh, sw, Cout)

        # Reorder to interleave scales with spatial dims:
        # want: (B, D, sd, H, sh, W, sw, Cout) -> (B, D*sd, H*sh, W*sw, Cout)
        x = x.permute(0, 1, 4, 2, 5, 3, 6, 7)     # (B, D, sd, H, sh, W, sw, Cout)
        x = x.reshape(B, D * s, H * s, W * s, Cout)

        return x

class Deconv3DStack(nn.Module):
    """
    Deconv3D 100% Linear+Shuffle (channel-last).
    Pas de ConvTranspose3D, pas de Conv3D.
    """
    def __init__(self, in_c: int, deconv_layers, final_c: int = 32):
        super().__init__()
        self.out_c = final_c

        # Exemple de pyramide de channels (à ajuster si besoin)
        self.layers = nn.Sequential(
            ChannelLastUpsample3d(in_c, 256, scale=2),   # x2
            nn.LayerNorm(256),
            nn.ReLU(),
            ChannelLastUpsample3d(256, 128, scale=2),   # x2
            nn.ReLU(),
            ChannelLastUpsample3d(128, final_c, scale=2),    # x2
            # nn.ReLU(inplace=True),
            # ChannelLastUpsample3d(128, final_c, scale=2), # x2
        )

        self.apply(self._init_weights)

    @staticmethod
    def _init_weights(m):
        if isinstance(m, nn.Linear):
            nn.init.trunc_normal_(m.weight, std=0.02)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0.0)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: (B, D, H, W, Cin) channel-last
        return self.layers(x)

# =========================================================
# MLP IMPLICITE & GATHERND (5D)
# =========================================================
class ImplicitMLP(nn.Module):
    """
    Head implicite (Linear only).
    - features: (B, N, Cfeat)
    - points_norm: (B, N, 3)
    - output logits: (B, N, NUM_CLASSES)
    """
    def __init__(self, in_dim_feat: int, hidden_dim: int = 64, num_classes: int = NUM_CLASSES):
        super().__init__()
        self.hidden_dim_intern=hidden_dim+3

        self.preproFeatures = nn.Sequential(
            nn.Linear(in_dim_feat, hidden_dim, bias=True),
        )
        self.feat_fc = nn.Sequential(
            nn.LayerNorm(self.hidden_dim_intern),
            nn.Linear(self.hidden_dim_intern, self.hidden_dim_intern, bias=True),
            nn.ReLU(inplace=True),
            nn.Linear(self.hidden_dim_intern, self.hidden_dim_intern, bias=True),
            nn.ReLU(inplace=True),
            nn.Linear(self.hidden_dim_intern, num_classes, bias=True),
        )

    def forward(self, xyz: torch.Tensor, features: torch.Tensor) -> torch.Tensor:
        # points_norm: (B,N,3), features: (B,N,Cfeat)
        features = self.preproFeatures(features)
        features = torch.cat([features, xyz], dim=-1)
        logits = self.feat_fc(features)
        return logits

# =========================================================
# MODELE GLOBAL
# =========================================================
class SurroundOccBinary(nn.Module):
    def __init__(self, backbone_name="mobilenetv3_small_100", dim_BiFPN=128, dim=768, cams=8, query_voxel=(16, 16, 4),
                spatial_heads=12, spatial_layers=4, pos_drop=0.1, moe_experts=4, moe_capacity=1.25,
                temporal_heads=6, temporal_layers=2, temporal_drop=0.1, deconv_layers=4,
                backbone=None, max_points_per_forward=100_000, in_dim_feat=128):
        super().__init__()
        self.register_buffer('roi_min', torch.tensor([-16.0, -16.0, -2.0]).view(1, 1, 3))
        self.register_buffer('roi_max', torch.tensor([ 16.0,  16.0,  8.0]).view(1, 1, 3))
        self.roi_range = self.roi_max - self.roi_min

        self.backbone = timm.create_model("resnet10t", pretrained=True, features_only=True)
        backbone_chs = self.backbone.feature_info.channels()
        self.bifpn = BiFPN({"p2": backbone_chs[1], "p3": backbone_chs[2], "p4": backbone_chs[3]}, dim=dim_BiFPN)

        self.dim = dim
        self.positional_transformer = PositionalTransformer3D(
            cams=cams, query_voxel=query_voxel, spatial_dim=dim, spatial_heads=spatial_heads, spatial_layers=spatial_layers, drop=pos_drop
        )

        self.deconv3d = Deconv3DStack(dim, deconv_layers, final_c=32)
        self.mlp_head = ImplicitMLP(in_dim_feat=32, hidden_dim=128)

        self.register_buffer('scene_min', torch.tensor([-16.0, -16.0, -2.0]))
        self.register_buffer('scene_max', torch.tensor([ 16.0,  16.0,  6.0]))
        self.register_buffer('scene_range', self.scene_max - self.scene_min)

    def extract_feats(self, imgs):
        B, Cams, C, H, W = imgs.shape
        x = imgs.view(B * Cams, C, H, W)
        feats = self.backbone(x)
        f = self.bifpn({"p2": feats[1], "p3": feats[2], "p4": feats[3]})

        def reshape(t):
            return t.view(B, Cams, t.shape[1], t.shape[2], t.shape[3])

        return {k: reshape(v) for k, v in f.items()}

    def query_points_static(self, latent_dense: torch.Tensor, points_xyz_m: torch.Tensor) -> torch.Tensor:
        """
        Gather ND direct sur volume channel-last (B, D, H, W, C) sans flatten.
        latent_dense : (B, D, H, W, C)
        points_xyz_m : (B, N, 3) en mètres
        """
        B, D, H, W, C = latent_dense.shape
        N = points_xyz_m.shape[1]

        # Normalisation ROI -> [0,1)
        offset = points_xyz_m - self.scene_min  # (B,N,3)
        norm_pos = offset / self.scene_range
        # norm_pos = torch.clamp(norm_pos, 0.0, 0.9999)

        ix = (norm_pos[..., 0] * W).int()  # (B,N)
        iy = (norm_pos[..., 1] * H).int()
        iz = (norm_pos[..., 2] * D).int()

        # Indices ND (B,N) -> broadcast pour gather 4D
        b_idx = torch.arange(B, device=latent_dense.device).view(B, 1).expand(B, N)  # (B,N)

        # Gather direct: latent_dense[b, z, y, x, :] => (B,N,C)
        features = latent_dense[b_idx, iz, iy, ix, :]  # (B,N,C)
        logits = self.mlp_head(norm_pos, features)  # (B,N,7)
        return logits

    def forward(self, imgs, points_xyz_m):
        pyramids = self.extract_feats(imgs)

        # (B, C, qx, qy, qz)
        latent_coarse = self.positional_transformer(pyramids)

        latent_dense = self.deconv3d(latent_coarse)  # (B, D, H, W, Cout)

        logits = self.query_points_static(latent_dense, points_xyz_m)  # (B,N,7)
        # Pour training CE vous voulez (B,7,N)
        return logits.permute(0, 2, 1).contiguous()



# =========================================================
# TRAINING HELPERS
# =========================================================
def forward_model_chunked_training(model, imgs, pts_xyz_m, n_points_fixed=N_POINTS_FIXED):
    B, N, _ = pts_xyz_m.shape
    if N == n_points_fixed:
        return model(imgs, pts_xyz_m)

    logits_chunks = []
    for start in range(0, N, n_points_fixed):
        end = min(N, start + n_points_fixed)
        n_chunk_reel = end - start
        pts_chunk = pts_xyz_m[:, start:end, :]
        if n_chunk_reel < n_points_fixed:
            pad = torch.zeros(B, n_points_fixed - n_chunk_reel, 3, device=pts_xyz_m.device, dtype=pts_xyz_m.dtype)
            pts_input = torch.cat([pts_chunk, pad], dim=1)
        else:
            pts_input = pts_chunk

        logits_pad = model(imgs, pts_input)
        logits_chunks.append(logits_pad[:, :, :n_chunk_reel])

    return torch.cat(logits_chunks, dim=2)

def train_one_epoch_on_loader(model, loader, opt, sched, device, args, class_weights):
    model.train()
    run_loss = 0.0
    # scaler = torch.amp.GradScaler("cuda", enabled=False)

    accum_steps = max(1, getattr(args, "grad_accum_steps", 1))
    opt.zero_grad(set_to_none=True)

    meter = IterMeter(total_iters=len(loader), window=50)
    log_every = getattr(args, "log_every", 20)

    for it, (imgs, pts_xyz_m, labels, mask, metas) in enumerate(loader, start=1):
        imgs, pts_xyz_m, labels, mask = imgs.to(device), pts_xyz_m.to(device), labels.to(device), mask.to(device)

        with torch.amp.autocast("cuda", dtype=torch.bfloat16):
            logits = forward_model_chunked_training(model, imgs, pts_xyz_m, n_points_fixed=N_POINTS_FIXED)
            loss = F.cross_entropy(
                logits.permute(0, 2, 1).reshape(-1, NUM_CLASSES)[mask.reshape(-1)],
                labels.reshape(-1)[mask.reshape(-1)].long(),
                weight=class_weights,
                ignore_index=255
            )

        run_loss += loss.item()
        # loss_scaled = loss / accum_steps
        (loss / accum_steps).backward()

        if (it % accum_steps == 0) or (it == len(loader)):
            # scaler.unscale_(opt)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            opt.step()
            # scaler.update()
            sched.step()
            opt.zero_grad(set_to_none=True)

        # metering
        meter.update(1)

        if (it % log_every == 0) or (it == 1) or (it == len(loader)):
            it_s, eta_s = meter.stats()
            print(
                f" it {it:04d}/{len(loader):04d}"
                f" | loss {loss.item():.4f}"
                f" | {it_s:.2f} it/s"
                f" | ETA {_fmt_time(eta_s)}"
            )

    return run_loss / max(1, len(loader))


@torch.no_grad()
def evaluate_on_loader(model, loader, device, args, out_dir, class_weights, prefix):
    model.eval()
    run_loss, correct, total = 0.0, 0, 0
    viz_done = False

    meter = IterMeter(total_iters=len(loader), window=50)
    log_every = getattr(args, "log_every", 50)

    for it, (imgs, pts_xyz_m, labels, mask, metas) in enumerate(loader, start=1):
        imgs, pts_xyz_m, labels, mask = imgs.to(device), pts_xyz_m.to(device), labels.to(device), mask.to(device)

        with torch.amp.autocast("cuda", dtype=torch.bfloat16):
            logits = forward_model_chunked_training(model, imgs, pts_xyz_m, n_points_fixed=N_POINTS_FIXED)
            logits_valid = logits.permute(0, 2, 1).reshape(-1, NUM_CLASSES)[mask.reshape(-1)]
            labels_valid = labels.reshape(-1)[mask.reshape(-1)].long()

            loss = F.cross_entropy(logits_valid, labels_valid, weight=class_weights, ignore_index=255)
            run_loss += loss.item()

            preds = logits_valid.argmax(dim=-1)
            correct += (preds == labels_valid).sum().item()
            total += len(labels_valid)

            if not viz_done:
                # Déplace le .cpu() ici pour ne pas bloquer la A100
                preds0 = logits[0].argmax(dim=0).detach().cpu().numpy()
                pts0 = pts_xyz_m[0].detach().cpu().numpy()
                save_3d_cubes(pts0, labels_valid, os.path.join(out_dir, f"{prefix}_true_viz.png"))
                save_3d_cubes(pts0, preds0, os.path.join(out_dir, f"{prefix}_viz.png"))
                viz_done = True

        meter.update(1)
        if (it % log_every == 0) or (it == 1) or (it == len(loader)):
            it_s, eta_s = meter.stats()
            print(
                f" val {it:04d}/{len(loader):04d}"
                f" | loss {loss.item():.4f}"
                f" | {it_s:.2f} it/s"
                f" | ETA {_fmt_time(eta_s)}"
            )

    return run_loss / max(1, len(loader)), correct / max(1, total)

# =========================================================
# QAT SETUP (DESACTIVE)
# =========================================================
def setup_qat_lss(model):
    return model

# =========================================================
# MAIN
# =========================================================
def build_parser():
    p = argparse.ArgumentParser()
    p.add_argument("--data_root", type=str, default=r"/content/DataSet/finalDataset/finalDataset")
    p.add_argument("--out_dir", type=str, default=r"/content/drive/MyDrive/OccMLP/Model_V9/")
    p.add_argument("--epochs", type=int, default=20)
    p.add_argument("--batch_size", type=int, default=32)
    p.add_argument("--lr", type=float, default=1e-4)
    p.add_argument("--weight_decay", type=float, default=0.05)
    p.add_argument("--num_workers", type=int, default=min(8, os.cpu_count() or 4))
    p.add_argument("--seed", type=int, default=42)
    p.add_argument("--grad_accum_steps", type=int, default=1)
    p.add_argument("--empty_ratio", type=float, default=0.0)

    # transformer
    p.add_argument("--dim", type=int, default=512)
    p.add_argument("--spatial_heads", type=int, default=8)
    p.add_argument("--spatial_layers", type=int, default=8)

    p.add_argument("--resume", type=str, default="")
    p.add_argument("--backbone", type=str, default="mobilenetv3_small_100")

    p.add_argument("--log_every", type=int, default=20)
    return p

def main():
    args = build_parser().parse_args([])
    enable_flash_attention()
    os.makedirs(args.out_dir, exist_ok=True)
    set_seed(args.seed)

    model = SurroundOccBinary(backbone_name=args.backbone, dim=args.dim, spatial_heads=args.spatial_heads,
                              spatial_layers=args.spatial_layers, cams=len(CAM_NAMES)).to(DEVICE)

    # QAT désactivé
    model = setup_qat_lss(model)
    print("Modèle initialisé. device :")
    print(DEVICE)
    if args.resume and os.path.isfile(args.resume):
        start_epoch, best_acc = load_checkpoint(args.resume, model, map_location=DEVICE)
        print(f"🔁 Reprise: Epoch {start_epoch}, Acc {best_acc:.4f}")
    else:
        start_epoch, best_acc = 0, 0.0

    opt = torch.optim.AdamW(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)


    class_weights = torch.tensor([
      0.5,   # Empty
      0.7,   # Unknown
      1.0,   # Road
      1.2,   # Sidewalk
      1.5,   # Vegetation
      3.0,   # Vehicles   <<<<< CRITIQUE
      2.0,   # Occupied   <<<<< CRITIQUE
    ], device=DEVICE)


    root = Path(args.data_root)
    subdirs = sorted([d for d in root.iterdir() if d.is_dir()])
    if not subdirs:
        raise FileNotFoundError(f"Pas de sous-dossiers dans {root}")

    all_train_dls = []
    all_val_dls = []
    for sd in subdirs:
        ds_tr = ImplicitOccDataset(str(sd), split="train")
        ds_va = ImplicitOccDataset(str(sd), split="val")

        # Utilisez persistent_workers=True pour garder les processus en vie
        all_train_dls.append(DataLoader(ds_tr, batch_size=args.batch_size, shuffle=True,
                                        num_workers=args.num_workers, collate_fn=collate_implicit,
                                        persistent_workers=True, pin_memory=True))
        all_val_dls.append(DataLoader(ds_va, batch_size=args.batch_size,
                                      num_workers=args.num_workers, collate_fn=collate_implicit,
                                      persistent_workers=True, pin_memory=True))
    # Ajoute ceci : un scheduler OneCycle
    iters_per_epoch = sum(len(dl) for dl in all_train_dls)
    total_steps = args.epochs * iters_per_epoch

    sched = torch.optim.lr_scheduler.OneCycleLR(
        opt, max_lr=args.lr * 2,
        total_steps=total_steps,
        pct_start=0.2,
        div_factor=10,
        final_div_factor=100
    )

    for ep in range(start_epoch, args.epochs):
        for dl_tr, dl_va, sd_name in zip(all_train_dls, all_val_dls, [s.name for s in subdirs]):
            # ds_tr = ImplicitOccDataset(str(sd), split="train", seed=args.seed)
            # ds_va = ImplicitOccDataset(str(sd), split="val", seed=args.seed)
            # dl_tr = DataLoader(ds_tr, batch_size=args.batch_size, shuffle=True, num_workers=args.num_workers, collate_fn=collate_implicit)
            # dl_va = DataLoader(ds_va, batch_size=args.batch_size, shuffle=False, num_workers=args.num_workers, collate_fn=collate_implicit)

            train_loss = train_one_epoch_on_loader(model, dl_tr, opt, sched, DEVICE, args, class_weights)
            val_loss, val_acc = evaluate_on_loader(model, dl_va, DEVICE, args, args.out_dir, class_weights, sd.name)
            print(f"[epoch : {ep} : {sd.name}] Train Loss: {train_loss:.4f} Val Loss: {val_loss:.4f} Acc: {val_acc*100:.2f}%")

        if val_acc > best_acc:
            best_acc = val_acc
            save_checkpoint(os.path.join(args.out_dir, "best.ckpt"), model, opt, ep + 1, best_acc)
        save_checkpoint(os.path.join(args.out_dir, "last.ckpt"), model, opt, ep + 1, best_acc)

if __name__ == "__main__":
    main()

ℹ️ QAT désactivé : couches nn standard.


/usr/local/lib/python3.12/dist-packages/torch/backends/cuda/__init__.py:131: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  return torch._C._get_cublas_allow_tf32()
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will b

✅ PE Fourier généré : torch.Size([1, 1024, 512]) (Sigma=8)
Modèle initialisé. device :
cuda
[train] frames: 85000 | root=/content/DataSet/finalDataset/finalDataset/100k
[train] Subsampling: target between 45000 and 60000 pts.
[val] frames: 15000 | root=/content/DataSet/finalDataset/finalDataset/100k
[val] Subsampling: target between 45000 and 60000 pts.
 it 0001/2657 | loss 1.9251 | 0.03 it/s | ETA 28:07:46
 it 0020/2657 | loss 1.8462 | 0.38 it/s | ETA 01:54:30
 it 0040/2657 | loss 1.7786 | 0.59 it/s | ETA 01:13:21
 it 0060/2657 | loss 1.6901 | 1.32 it/s | ETA 32:49
 it 0080/2657 | loss 1.6096 | 1.32 it/s | ETA 32:34
 it 0100/2657 | loss 1.5232 | 1.32 it/s | ETA 32:18
 it 0120/2657 | loss 1.4239 | 1.32 it/s | ETA 32:02
 it 0140/2657 | loss 1.3792 | 1.32 it/s | ETA 31:47
 it 0160/2657 | loss 1.2612 | 1.32 it/s | ETA 31:32
 it 0180/2657 | loss 1.1477 | 1.32 it/s | ETA 31:17
 it 0200/2657 | loss 1.1192 | 1.32 it/s | ETA 31:01
 it 0220/2657 | loss 1.0166 | 1.32 it/s | ETA 30:46
 it 0240/26

KeyboardInterrupt: 

In [ ]:
import os
import re

# 1. Configuration
directory = '/content/DataSet/finalDataset/100k/images'  # '.' signifie le dossier où se trouve le script
limit_to_keep = 96877
dry_run = False  # METTRE À False POUR RÉELLEMENT SUPPRIMER

def clean_frames():
    count = 0
    # Pattern : cherche 'frame_', capture les chiffres (\d+), puis s'arrête au '_' suivant
    pattern = re.compile(r"frame_(\d+)_")

    print(f"--- Analyse du dossier : {os.path.abspath(directory)} ---")

    if dry_run:
        print("[MODE TEST] Aucun fichier ne sera supprimé. Modifiez 'dry_run = False' pour appliquer.\n")

    for filename in os.listdir(directory):
        match = pattern.search(filename)

        if match:
            # On extrait le numéro et on le convertit en entier
            frame_number = int(match.group(1))

            # Condition de suppression : tout ce qui est strictement supérieur à 96877
            if frame_number > limit_to_keep:
                file_path = os.path.join(directory, filename)

                if dry_run:
                    print(f"[TEST] Suppression de : {filename} (ID: {frame_number})")
                else:
                    try:
                        os.remove(file_path)
                        print(f"Supprimé : {filename}")
                    except Exception as e:
                        print(f"Erreur sur {filename} : {e}")

                count += 1

    print(f"\nTerminé. Total de fichiers à supprimer/supprimés : {count}")

if __name__ == "__main__":
    clean_frames()

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Supprimé : frame_098533_front_left.jpg
Supprimé : frame_099310_back.jpg
Supprimé : frame_099336_front_left.jpg
Supprimé : frame_096992_front.jpg
Supprimé : frame_098624_back_right.jpg
Supprimé : frame_098986_front_left.jpg
Supprimé : frame_099608_back.jpg
Supprimé : frame_097475_right.jpg
Supprimé : frame_098627_back.jpg
Supprimé : frame_098676_left.jpg
Supprimé : frame_099118_left.jpg
Supprimé : frame_097790_front_left.jpg
Supprimé : frame_097285_left.jpg
Supprimé : frame_098484_back_left.jpg
Supprimé : frame_099991_back_left.jpg
Supprimé : frame_097456_left.jpg
Supprimé : frame_099919_front_left.jpg
Supprimé : frame_098290_front_left.jpg
Supprimé : frame_097990_back_left.jpg
Supprimé : frame_098314_back_left.jpg
Supprimé : frame_099932_back.jpg
Supprimé : frame_097170_back_left.jpg
Supprimé : frame_097448_back_left.jpg
Supprimé : frame_098558_back.jpg
Supprimé : frame_099266_front_right.jpg
Supprimé : frame

In [ ]:
import os
import re

# --- CONFIGURATION ---
DIRECTORY = '/content/DataSet/finalDataset/100k/points/'              # Dossier courant
START_INDEX = 96878          # Début de la zone à supprimer
END_INDEX = 98677            # Fin de la zone à supprimer
DRY_RUN = False               # True pour tester, False pour supprimer réellement

def cleanup_specific_range():
    # Regex pour capturer le numéro après 'frame_'
    pattern = re.compile(r"frame_(\d+)")
    deleted_count = 0

    print(f"🔍 Analyse du dossier : {os.path.abspath(DIRECTORY)}")
    if DRY_RUN:
        print("⚠️  MODE TEST ACTIVÉ : Rien ne sera supprimé.\n")

    # Liste des fichiers dans le dossier
    files = os.listdir(DIRECTORY)

    for filename in files:
        match = pattern.search(filename)

        if match:
            # Extraction du numéro de frame
            frame_num = int(match.group(1))

            # Vérification si le numéro est dans la plage cible
            if START_INDEX <= frame_num <= END_INDEX:
                file_path = os.path.join(DIRECTORY, filename)

                if DRY_RUN:
                    print(f"[TEST] À supprimer : {filename}")
                else:
                    try:
                        os.remove(file_path)
                        print(f"Supprimé : {filename}")
                    except OSError as e:
                        print(f"Erreur sur {filename} : {e}")

                deleted_count += 1

    print("-" * 30)
    if DRY_RUN:
        print(f"Action terminée (simulation). {deleted_count} fichiers auraient été supprimés.")
        print("Si la liste est correcte, passez 'DRY_RUN = False' dans le script.")
    else:
        print(f"Nettoyage terminé. {deleted_count} fichiers ont été supprimés.")

if __name__ == "__main__":
    cleanup_specific_range()

🔍 Analyse du dossier : /content/DataSet/finalDataset/100k/points
Supprimé : frame_097528.npz
Supprimé : frame_097284.npz
Supprimé : frame_097985.npz
Supprimé : frame_098575.npz
Supprimé : frame_096915.npz
Supprimé : frame_096935.npz
Supprimé : frame_097685.npz
Supprimé : frame_098050.npz
Supprimé : frame_098658.npz
Supprimé : frame_098591.npz
Supprimé : frame_097916.npz
Supprimé : frame_098011.npz
Supprimé : frame_098152.npz
Supprimé : frame_097723.npz
Supprimé : frame_098401.npz
Supprimé : frame_097205.npz
Supprimé : frame_097442.npz
Supprimé : frame_097388.npz
Supprimé : frame_097895.npz
Supprimé : frame_097127.npz
Supprimé : frame_097565.npz
Supprimé : frame_098583.npz
Supprimé : frame_098008.npz
Supprimé : frame_098241.npz
Supprimé : frame_098302.npz
Supprimé : frame_098280.npz
Supprimé : frame_098303.npz
Supprimé : frame_098072.npz
Supprimé : frame_098217.npz
Supprimé : frame_098067.npz
Supprimé : frame_097142.npz
Supprimé : frame_097796.npz
Supprimé : frame_097830.npz
Supprimé : 

In [ ]:
import os
import csv
from pathlib import Path
from PIL import Image, ImageFile, UnidentifiedImageError

# IMPORTANT: pour détecter les images tronquées/corrompues, il faut False
ImageFile.LOAD_TRUNCATED_IMAGES = False

VALID_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp", ".tif", ".tiff"}

def check_one_image(path: Path, expected_w: int, expected_h: int):
    issues = []
    meta = {"mode": None, "width": None, "height": None, "format": None}

    # 1) verify()
    try:
        with Image.open(path) as im:
            meta["format"] = im.format
            meta["mode"] = im.mode
            meta["width"], meta["height"] = im.size
            im.verify()
    except (UnidentifiedImageError, OSError, ValueError) as e:
        issues.append(f"verify_fail:{type(e).__name__}:{str(e)[:200]}")
        return False, issues, meta

    # 2) re-open + load()
    try:
        with Image.open(path) as im2:
            im2.load()
            meta["format"] = im2.format
            meta["mode"] = im2.mode
            meta["width"], meta["height"] = im2.size

            if meta["mode"] not in ("RGB", "RGBA", "L"):
                issues.append(f"unexpected_mode:{meta['mode']}")

            if (meta["width"] != expected_w) or (meta["height"] != expected_h):
                issues.append(f"bad_size:{meta['width']}x{meta['height']} expected:{expected_w}x{expected_h}")

    except (UnidentifiedImageError, OSError, ValueError) as e:
        issues.append(f"load_fail:{type(e).__name__}:{str(e)[:200]}")
        return False, issues, meta

    return (len(issues) == 0), issues, meta


def scan_images(images_root: str, expected_w=512, expected_h=384,
                csv_out="/content/image_check_report.csv",
                bad_dir="", limit=0):
    root = Path(images_root)
    if not root.exists():
        raise FileNotFoundError(root)

    files = [p for p in root.rglob("*") if p.is_file() and p.suffix.lower() in VALID_EXTS]
    files.sort()
    if limit and limit > 0:
        files = files[:limit]

    print(f"Scanning {len(files)} images under: {root}")

    bad_dir_path = Path(bad_dir) if bad_dir else None
    if bad_dir_path:
        bad_dir_path.mkdir(parents=True, exist_ok=True)

    ok_count = bad_count = 0
    verify_fail = load_fail = size_mismatch = mode_weird = 0

    with open(csv_out, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["path", "ok", "format", "mode", "width", "height", "issues"])

        for i, img_path in enumerate(files, start=1):
            ok, issues, meta = check_one_image(img_path, expected_w, expected_h)

            if ok:
                ok_count += 1
            else:
                bad_count += 1

            for iss in issues:
                if iss.startswith("bad_size:"):
                    size_mismatch += 1
                elif iss.startswith("unexpected_mode:"):
                    mode_weird += 1
                elif iss.startswith("verify_fail:"):
                    verify_fail += 1
                elif iss.startswith("load_fail:"):
                    load_fail += 1

            w.writerow([
                str(img_path),
                int(ok),
                meta.get("format"),
                meta.get("mode"),
                meta.get("width"),
                meta.get("height"),
                "|".join(issues),
            ])

            if bad_dir_path and (not ok):
                # copie brute
                dst = bad_dir_path / img_path.name
                try:
                    dst.write_bytes(img_path.read_bytes())
                except Exception:
                    pass

            if (i % 2000) == 0:
                print(f"  {i}/{len(files)} scanned... bad={bad_count}")

    print("\n=== DONE ===")
    print(f"OK:  {ok_count}")
    print(f"BAD: {bad_count}")
    print(f" - verify_fail:     {verify_fail}")
    print(f" - load_fail:       {load_fail}")
    print(f" - size_mismatch:   {size_mismatch}")
    print(f" - unexpected_mode: {mode_weird}")
    print(f"CSV report: {csv_out}")
    if bad_dir_path:
        print(f"Bad images copied to: {bad_dir_path}")


# >>>>>> ICI: mets ton chemin images <<<<<<
scan_images(
    images_root="/content/DataSet/finalDataset/100k/images",  # <-- adapte
    expected_w=512,
    expected_h=384,
    csv_out="/content/image_check_report.csv",
    bad_dir="/content/bad_images",  # "" si tu ne veux pas copier
    limit=0  # mets 2000 pour tester vite
)


Scanning 775024 images under: /content/DataSet/finalDataset/100k/images
  2000/775024 scanned... bad=0
  4000/775024 scanned... bad=0
  6000/775024 scanned... bad=0
  8000/775024 scanned... bad=0
  10000/775024 scanned... bad=0
  12000/775024 scanned... bad=0
  14000/775024 scanned... bad=0
  16000/775024 scanned... bad=0
  18000/775024 scanned... bad=0
  20000/775024 scanned... bad=0
  22000/775024 scanned... bad=0
  24000/775024 scanned... bad=0
  26000/775024 scanned... bad=0
  28000/775024 scanned... bad=0
  30000/775024 scanned... bad=0
  32000/775024 scanned... bad=0
  34000/775024 scanned... bad=0
  36000/775024 scanned... bad=0
  38000/775024 scanned... bad=0
  40000/775024 scanned... bad=0
  42000/775024 scanned... bad=0
  44000/775024 scanned... bad=0
  46000/775024 scanned... bad=0
  48000/775024 scanned... bad=0
  50000/775024 scanned... bad=0
  52000/775024 scanned... bad=0
  54000/775024 scanned... bad=0
  56000/775024 scanned... bad=0
  58000/775024 scanned... bad=0
  60

In [ ]:
import time
while(True):
  time.sleep(10)

In [ ]:
!ls /content/DataSet/finalDataset/100k/points

frame_000000.npz  frame_024635.npz  frame_049270.npz  frame_073905.npz
frame_000001.npz  frame_024636.npz  frame_049271.npz  frame_073906.npz
frame_000002.npz  frame_024637.npz  frame_049272.npz  frame_073907.npz
frame_000003.npz  frame_024638.npz  frame_049273.npz  frame_073908.npz
frame_000004.npz  frame_024639.npz  frame_049274.npz  frame_073909.npz
frame_000005.npz  frame_024640.npz  frame_049275.npz  frame_073910.npz
frame_000006.npz  frame_024641.npz  frame_049276.npz  frame_073911.npz
frame_000007.npz  frame_024642.npz  frame_049277.npz  frame_073912.npz
frame_000008.npz  frame_024643.npz  frame_049278.npz  frame_073913.npz
frame_000009.npz  frame_024644.npz  frame_049279.npz  frame_073914.npz
frame_000010.npz  frame_024645.npz  frame_049280.npz  frame_073915.npz
frame_000011.npz  frame_024646.npz  frame_049281.npz  frame_073916.npz
frame_000012.npz  frame_024647.npz  frame_049282.npz  frame_073917.npz
frame_000013.npz  frame_024648.npz  frame_049283.npz  frame_073918.npz
frame_

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
One-script: load FP32 checkpoint -> ModelOpt INT8 quantize + calib -> re-init geo cache AFTER quantizers
-> (optional) QAT training -> export ONNX Q/DQ

Key fix:
- geo cache init is called AFTER mtq.quantize(), so geo_mlp quantizers get amax.
- calib loader uses num_workers=0 to avoid Colab "Bad file descriptor".
"""

import os
import time
import copy
import math
import argparse
from pathlib import Path

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset

import modelopt.torch.quantization as mtq
from modelopt.torch._deploy.utils import OnnxBytes, get_onnx_bytes_and_metadata

# ---------------------------------------------------------------------
# NOTE:
# This script assumes the following are already defined/imported in your environment:
# - DEVICE, IMAGE_SIZE, CAM_NAMES, DEFAULT_CAM_PLACEMENTS
# - N_POINTS_FIXED
# - ImplicitOccDataset, collate_implicit
# - SurroundOccBinary, CameraRig
# - enable_flash_attention, set_seed
# - load_checkpoint, save_checkpoint
# - forward_model_chunked_training
# - save_3d_cubes + evaluate/train fns if you keep training
# ---------------------------------------------------------------------


def build_parser():
    p = argparse.ArgumentParser()

    p.add_argument("--data_root", type=str, default="/content/DataSet")
    p.add_argument("--out_dir", type=str, default="/content/drive/MyDrive/OccMLP/Model_V8/QAT")
    p.add_argument("--epochs", type=int, default=1)
    p.add_argument("--batch_size", type=int, default=16)
    p.add_argument("--lr", type=float, default=1e-4)
    p.add_argument("--weight_decay", type=float, default=0.05)
    p.add_argument("--num_workers", type=int, default=4)
    p.add_argument("--seed", type=int, default=42)
    p.add_argument("--grad_accum_steps", type=int, default=1)
    p.add_argument("--empty_ratio", type=float, default=0.0)

    # model
    p.add_argument("--dim", type=int, default=384)
    p.add_argument("--qvolx", type=int, default=8)
    p.add_argument("--qvoly", type=int, default=8)
    p.add_argument("--qvolz", type=int, default=4)
    p.add_argument("--pos_heads", type=int, default=6)
    p.add_argument("--pos_layers", type=int, default=2)
    p.add_argument("--pos_drop", type=float, default=0.1)
    p.add_argument("--moe_experts", type=int, default=2)
    p.add_argument("--moe_capacity", type=float, default=1.25)
    p.add_argument("--temp_heads", type=int, default=6)
    p.add_argument("--temp_layers", type=int, default=2)
    p.add_argument("--temp_drop", type=float, default=0.1)
    p.add_argument("--deconv_layers", type=int, default=4)
    p.add_argument("--max_points_per_forward", type=int, default=16384 * 8)
    p.add_argument("--resume", type=str, default="/content/drive/MyDrive/OccMLP/Model_V8/epoch_0001.ckpt")
    p.add_argument("--backbone", type=str, default="mobilenetv3_small_100")

    # ModelOpt
    p.add_argument(
        "--modelopt_quantize_cfg",
        type=str,
        default="INT8_SMOOTHQUANT_CFG",
        choices=mtq.config.choices,
    )
    p.add_argument(
        "--onnx_export_path",
        type=str,
        default="/content/drive/MyDrive/OccMLP/Model_V8/QAT/surroundocc_int8_Test_23.onnx",
    )
    p.add_argument("--calib_samples", type=int, default=500)

    return p


def build_calib_loader(args):
    root = Path(args.data_root)
    subdirs = sorted([d for d in root.iterdir() if d.is_dir()])
    if not subdirs:
        raise FileNotFoundError(f"Pas de sous-dossiers dans {root}")
    calib_dir = subdirs[0]
    print(f"[calib] Utilisation du dossier {calib_dir} pour la calibration")

    ds_full = ImplicitOccDataset(
        str(calib_dir),
        split="train",
        val_ratio=0.15,
        seed=args.seed,
        expect_hw=IMAGE_SIZE,
        normalize=True,
        empty_ratio=args.empty_ratio,
    )
    n_calib = min(args.calib_samples, len(ds_full))
    ds_calib = Subset(ds_full, list(range(n_calib)))
    print(f"[calib] frames totales: {len(ds_full)} | on ne prend que {n_calib}")

    # IMPORTANT: num_workers=0 for Colab stability
    dl = DataLoader(
        ds_calib,
        batch_size=min(2, args.batch_size),
        shuffle=False,
        num_workers=0,
        pin_memory=False,
        collate_fn=collate_implicit,
        drop_last=False,
    )
    return dl


@torch.no_grad()
def forward_loop_for_calib(model, calib_loader, device):
    model.eval()
    for imgs, pts_xyz_m, labels, mask, metas in calib_loader:
        imgs = imgs.to(device, non_blocking=True)
        pts_xyz_m = pts_xyz_m.to(device, non_blocking=True)
        _ = forward_model_chunked_training(model, imgs, pts_xyz_m, n_points_fixed=N_POINTS_FIXED)


def check_quantizers(model):
    bad = []
    enabled = 0
    for name, module in model.named_modules():
        for qn in ("input_quantizer", "weight_quantizer", "output_quantizer"):
            if hasattr(module, qn):
                q = getattr(module, qn)
                if getattr(q, "is_enabled", False):
                    enabled += 1
                    a = getattr(q, "amax", None)
                    if a is None:
                        bad.append((name, qn, "amax None"))
                    elif torch.is_tensor(a):
                        if (not torch.isfinite(a).all()) or (a <= 0).any():
                            bad.append((name, qn, "non-finite or <=0"))
                    else:
                        fa = float(a)
                        if (not math.isfinite(fa)) or (fa <= 0):
                            bad.append((name, qn, "non-finite or <=0"))

    print(f"[Quant] quantizers enabled: {enabled}")
    print(f"[Quant] bad quantizers: {len(bad)}")
    for b in bad[:40]:
        print(" ", b)
    if bad:
        raise RuntimeError("Calibration invalide: amax incorrects (None / non-finite / <=0).")


def export_onnx_qdq(model, out_path: str):
    os.makedirs(os.path.dirname(out_path) or ".", exist_ok=True)
    export_model = model.to("cpu").eval()

    dummy_imgs = torch.zeros(1, len(CAM_NAMES), 3, *IMAGE_SIZE, dtype=torch.float32)
    dummy_pts = torch.zeros(1, N_POINTS_FIXED, 3, dtype=torch.float32)

    print(f"[ONNX] Export Q/DQ vers: {out_path}")
    onnx_bytes, _ = get_onnx_bytes_and_metadata(
        export_model,
        (dummy_imgs, dummy_pts),
        model_name="SurroundOccBinary_QAT",
        onnx_opset=18,
    )
    out_dir = os.path.dirname(out_path) or "."
    onnx_obj = OnnxBytes.from_bytes(onnx_bytes)
    onnx_obj.write_to_disk(out_dir, clean_dir=False)

    generated = os.path.join(out_dir, "SurroundOccBinary_QAT.onnx")
    if os.path.exists(generated) and generated != out_path:
        os.replace(generated, out_path)

    print("[ONNX] Export terminé.")


def main():
    args = build_parser().parse_args([])  # replace [] by None for CLI
    enable_flash_attention()
    os.makedirs(args.out_dir, exist_ok=True)
    set_seed(args.seed)

    rig = CameraRig(
        DEFAULT_CAM_PLACEMENTS,
        img_h=IMAGE_SIZE[0],
        img_w=IMAGE_SIZE[1],
    ).to(DEVICE)

    model = SurroundOccBinary(
        backbone_name=args.backbone,
        dim=args.dim,
        cams=len(CAM_NAMES),
        query_voxel=(args.qvolx, args.qvoly, args.qvolz),
        pos_heads=args.pos_heads,
        pos_layers=args.pos_layers,
        pos_drop=args.pos_drop,
        moe_experts=args.moe_experts,
        moe_capacity=args.moe_capacity,
        temporal_heads=args.temp_heads,
        temporal_layers=args.temp_layers,
        temporal_drop=args.temp_drop,
        deconv_layers=args.deconv_layers,
        rig=rig,
        max_points_per_forward=args.max_points_per_forward,
    ).to(DEVICE)

    # ---- load checkpoint (FP32) ----
    if args.resume and os.path.isfile(args.resume):
        print(f"[CKPT] Loading: {args.resume}")
        _start_epoch, _best_acc = load_checkpoint(args.resume, model, optimizer=None, map_location=DEVICE)
    else:
        print("[CKPT] No resume checkpoint found; continuing with random init.")

    # ---- create calib loader ----
    calib_loader = build_calib_loader(args)

    def forward_loop(m):
        forward_loop_for_calib(m, calib_loader, DEVICE)

    # ---- build quant config ----
    print(f"[ModelOpt] Quant config preset = {args.modelopt_quantize_cfg}")
    print("   Config  ")

    MY_INT8_CFG = copy.deepcopy(mtq.INT8_DEFAULT_CFG)
    qc = MY_INT8_CFG["quant_cfg"]

    ##### BACKBONE optimization ####

    qc["backbone.layer2.0.downsample.2.output_quantizer"] = {"enable": True, "num_bits": 8}
    qc["backbone.layer3.0.downsample.2.output_quantizer"] = {"enable": True, "num_bits": 8}



    ###### BiFPN optimization ######
    qc["bifpn.lat_p2.bn.output_quantizer"] = {"enable": True, "num_bits": 8}
    qc["bifpn.lat_p3.bn.output_quantizer"] = {"enable": True, "num_bits": 8}
    # qc["bifpn.up_p3.1.output_quantizer"] = {"enable": True, "num_bits": 8}
    qc["bifpn.lat_p4.1.output_quantizer"] = {"enable": True, "num_bits": 8}
    qc["bifpn.up_p4.0.output_quantizer"] = {"enable": True, "num_bits": 8}
    qc["bifpn.td_p3.2.output_quantizer"] = {"enable": True, "num_bits": 8}
    qc["bifpn.bu_p3.2.output_quantizer"] = {"enable": True, "num_bits": 8}
    qc["bifpn.td_p2.2.output_quantizer"] = {"enable": True, "num_bits": 8}

    qc["bifpn.up_p3.0.output_quantizer"] = {"enable": True, "num_bits": 8}

    # 1. Branche Latérale (xf2 sur l'image)
    # On active la quantification en sortie des couches latérales (p2, p3, p4)
    # Vos logs : bifpn.lat_p2.conv.output_quantizer = disabled -> On passe à True

    # qc["deconv3d.layers.4.bn.output_quantizer"] = {"enable": True, "num_bits": 8}


    model = mtq.quantize(model, MY_INT8_CFG, forward_loop)
    mtq.print_quant_summary(model)
    # =========================================================
    # FIX: re-init geo cache AFTER quantizers are inserted
    # so geo_mlp.fc1/fc2 quantizers get executed and get amax.
    # =========================================================


    # # Re-run calibration once, to populate amax for geo_mlp etc.
    # print("[ModelOpt] Re-calibration after geo cache init…")
    # with torch.no_grad():
    #     forward_loop(model)

    # ---- verify calibration ----
    # check_quantizers(model)

    # ---- export ONNX Q/DQ ----
    if args.onnx_export_path:
        export_onnx_qdq(model, args.onnx_export_path)

    print("✅ Done.")


if __name__ == "__main__":
    main()


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# QAT


"""
One-script: load FP32 checkpoint -> ModelOpt INT8 PTQ quantize + calib -> export ONNX Q/DQ

- Compatible avec le "nouveau" SurroundOcc (Linear standard, pas de QAT/LSS).
- Pas de geo-cache / re-init : on quantize + calib + export.
- Calibration loader num_workers=0 (Colab stable).

Pré-requis dans l'environnement (déjà définis/importés) :
- DEVICE, IMAGE_SIZE, CAM_NAMES, DEFAULT_CAM_PLACEMENTS
- N_POINTS_FIXED
- ImplicitOccDataset, collate_implicit
- SurroundOccBinary, CameraRig
- enable_flash_attention, set_seed
- load_checkpoint
- forward_model_chunked_training
"""

import os
import time
import copy
import math
import argparse
from pathlib import Path
import os
import math
import random
import argparse
from pathlib import Path
from typing import List, Tuple

import numpy as np
from PIL import Image, ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import timm
import matplotlib
import time
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

from torch.backends.cuda import sdp_kernel
import torch._inductor.config as inductor_config  # noqa: F401

import torch
from torch.utils.data import DataLoader, Subset

import modelopt.torch.quantization as mtq
from modelopt.torch._deploy.utils import OnnxBytes, get_onnx_bytes_and_metadata
from torch.utils.data import DataLoader, Subset

import modelopt.torch.quantization as mtq
from modelopt.torch._deploy.utils import OnnxBytes, get_onnx_bytes_and_metadata

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# ---------------------------------------------------------------------
# ARGPARSE
# ---------------------------------------------------------------------
def build_parser():
    p = argparse.ArgumentParser()

    # data / io
    p.add_argument("--data_root", type=str, default="/content/DataSet")
    p.add_argument("--out_dir", type=str, default="/content/drive/MyDrive/OccMLP/Model_V9/QAT")
    p.add_argument("--batch_size", type=int, default=20)
    p.add_argument("--seed", type=int, default=42)
    p.add_argument("--empty_ratio", type=float, default=0.0)
    p.add_argument("--resume", type=str, default="/content/drive/MyDrive/OccMLP/Model_V9/best.ckpt")
    p.add_argument("--epochs", type=int, default=6)
    p.add_argument("--weight_decay", type=float, default=0.05)
    p.add_argument("--num_workers", type=int, default=min(8, os.cpu_count() or 4))


    # model
    p.add_argument("--dim", type=int, default=512)
    p.add_argument("--qvolx", type=int, default=8)
    p.add_argument("--qvoly", type=int, default=8)
    p.add_argument("--qvolz", type=int, default=4)
    p.add_argument("--spatial_heads", type=int, default=8)
    p.add_argument("--spatial_layers", type=int, default=8)
    p.add_argument("--pos_drop", type=float, default=0.1)
    p.add_argument("--deconv_layers", type=int, default=4)
    p.add_argument("--max_points_per_forward", type=int, default=16384 * 8)
    p.add_argument("--backbone", type=str, default="mobilenetv3_small_100")

    # ModelOpt PTQ
    p.add_argument(
        "--modelopt_quantize_cfg",
        type=str,
        default="INT8_DEFAULT_CFG",
        choices=getattr(mtq.config, "choices", None) or None,
        help="Preset ModelOpt (ex: INT8_DEFAULT_CFG, INT8_SMOOTHQUANT_CFG, ...)",
    )
    p.add_argument("--calib_samples", type=int, default=500)

    # export
    p.add_argument(
        "--onnx_export_path",
        type=str,
        default="/content/drive/MyDrive/OccMLP/Model_V9/QAT/surroundocc_int8_q_dq.onnx",
    )
    p.add_argument("--onnx_opset", type=int, default=20)

    return p


# ---------------------------------------------------------------------
# CALIBRATION LOADER
# ---------------------------------------------------------------------
def build_calib_loader(args):
    root = Path(args.data_root)
    subdirs = sorted([d for d in root.iterdir() if d.is_dir()])
    if not subdirs:
        raise FileNotFoundError(f"Pas de sous-dossiers dans {root}")

    calib_dir = subdirs[0]
    print(f"[calib] Utilisation du dossier: {calib_dir}")

    ds_full = ImplicitOccDataset(
        str(calib_dir),
        split="train",
        val_ratio=0.15,
        seed=args.seed,
        expect_hw=IMAGE_SIZE,
        normalize=True,
        empty_ratio=args.empty_ratio,
    )
    n_calib = min(args.calib_samples, len(ds_full))
    ds_calib = Subset(ds_full, list(range(n_calib)))
    print(f"[calib] frames totales: {len(ds_full)} | frames calib: {n_calib}")

    # IMPORTANT: num_workers=0 pour Colab stabilité
    dl = DataLoader(
        ds_calib,
        batch_size=min(2, args.batch_size),
        shuffle=False,
        num_workers=0,
        pin_memory=False,
        collate_fn=collate_implicit,
        drop_last=False,
    )
    return dl


@torch.no_grad()
def forward_loop_for_calib(model, calib_loader, device):
    model.eval()
    for imgs, pts_xyz_m, labels, mask, metas in calib_loader:
        imgs = imgs.to(device, non_blocking=True)
        pts_xyz_m = pts_xyz_m.to(device, non_blocking=True)
        _ = forward_model_chunked_training(model, imgs, pts_xyz_m, n_points_fixed=N_POINTS_FIXED)


def check_quantizers(model):
    """
    Sanity check: quantizers enabled + amax finite/positive.
    Utile si TensorRT te dit "no scaling factors detected".
    """
    bad = []
    enabled = 0
    for name, module in model.named_modules():
        for qn in ("input_quantizer", "weight_quantizer", "output_quantizer"):
            if hasattr(module, qn):
                q = getattr(module, qn)
                if getattr(q, "is_enabled", False):
                    enabled += 1
                    a = getattr(q, "amax", None)
                    if a is None:
                        bad.append((name, qn, "amax None"))
                    elif torch.is_tensor(a):
                        if (not torch.isfinite(a).all()) or (a <= 0).any():
                            bad.append((name, qn, "non-finite or <=0"))
                    else:
                        fa = float(a)
                        if (not math.isfinite(fa)) or (fa <= 0):
                            bad.append((name, qn, "non-finite or <=0"))

    print(f"[Quant] quantizers enabled: {enabled}")
    print(f"[Quant] bad quantizers: {len(bad)}")
    for b in bad[:50]:
        print(" ", b)
    if bad:
        raise RuntimeError("Calibration invalide: amax incorrects (None / non-finite / <=0).")


# ---------------------------------------------------------------------
# CONFIG PRESET + OVERRIDES
# ---------------------------------------------------------------------
def get_modelopt_preset(preset_name: str) -> dict:
    """
    Récupère un preset ModelOpt sans dépendre d'une API fragile.
    - si mtq.<PRESET> existe -> deepcopy
    - sinon fallback sur mtq.INT8_DEFAULT_CFG
    """
    return copy.deepcopy(mtq.INT8_DEFAULT_CFG)


def apply_custom_overrides(cfg: dict) -> dict:
    """
    Place ici tes règles custom de quant_cfg.
    NOTE: Utiliser 'enable' et 'num_bits' (pas learn_amax). :contentReference[oaicite:1]{index=1}
    """
    cfg = copy.deepcopy(mtq.INT8_DEFAULT_CFG)
    qc = cfg.get("quant_cfg", {})

    # Exemple: activer des output_quantizer spécifiques
    # (Si ces chemins ne matchent pas dans ton modèle, ce n’est pas bloquant.)
    qc["backbone.layer2.0.downsample.2.output_quantizer"] = {"enable": True, "num_bits": 8}
    qc["backbone.layer3.0.downsample.2.output_quantizer"] = {"enable": True, "num_bits": 8}

    qc["bifpn.lat_p2.bn.output_quantizer"] = {"enable": True, "num_bits": 8}
    qc["bifpn.lat_p3.bn.output_quantizer"] = {"enable": True, "num_bits": 8}
    qc["bifpn.lat_p4.1.output_quantizer"] = {"enable": True, "num_bits": 8}
    qc["bifpn.up_p4.0.output_quantizer"] = {"enable": True, "num_bits": 8}
    qc["bifpn.td_p3.2.output_quantizer"] = {"enable": True, "num_bits": 8}
    qc["bifpn.bu_p3.2.output_quantizer"] = {"enable": True, "num_bits": 8}
    qc["bifpn.td_p2.2.output_quantizer"] = {"enable": True, "num_bits": 8}
    qc["bifpn.up_p3.0.output_quantizer"] = {"enable": True, "num_bits": 8}

    cfg["quant_cfg"] = qc
    return cfg


# ---------------------------------------------------------------------
# EXPORT ONNX Q/DQ
# ---------------------------------------------------------------------
def export_onnx_qdq(model, out_path: str, onnx_opset: int):
    os.makedirs(os.path.dirname(out_path) or ".", exist_ok=True)
    export_model = model.to("cpu").eval()

    dummy_imgs = torch.zeros(1, len(CAM_NAMES), 3, 384, 512, dtype=torch.float32)
    dummy_pts = torch.zeros(1, N_POINTS_FIXED, 3, dtype=torch.float32)

    print(f"[ONNX] Export Q/DQ vers: {out_path}")
    onnx_bytes, _ = get_onnx_bytes_and_metadata(
        export_model,
        (dummy_imgs, dummy_pts),
        model_name="SurroundOccBinary_INT8_QDQ",
        onnx_opset=onnx_opset,
    )
    out_dir = os.path.dirname(out_path) or "."
    onnx_obj = OnnxBytes.from_bytes(onnx_bytes)
    onnx_obj.write_to_disk(out_dir, clean_dir=False)

    generated = os.path.join(out_dir, "SurroundOccBinary_INT8_QDQ.onnx")
    if os.path.exists(generated) and generated != out_path:
        os.replace(generated, out_path)

    print("[ONNX] Export terminé.")

def enable_qat_learning(model):
    """
    Active l'apprentissage des paramètres de quantification si nécessaire.
    Avec ModelOpt, après mtq.quantize, le modèle est déjà en mode 'fake quant'.
    On s'assure juste que le modèle est en mode train.
    """
    model.train()
    # On peut aussi spécifiquement débloquer les amax si ModelOpt le permet dans la config
    return model

# ---------------------------------------------------------------------
# MAIN
# ---------------------------------------------------------------------
def main():
    args = build_parser().parse_args([])  # remplace [] par None pour CLI
    enable_flash_attention()
    os.makedirs(args.out_dir, exist_ok=True)
    set_seed(args.seed)



    # ----- Build model (NOUVEAU SurroundOcc) -----
    model = SurroundOccBinary(backbone_name=args.backbone, dim=args.dim,
                              spatial_heads=args.spatial_heads, spatial_layers=args.spatial_layers,
                              cams=len(CAM_NAMES)).to(DEVICE)


    # ----- Load FP32 checkpoint -----
    if args.resume and os.path.isfile(args.resume):
        print(f"[CKPT] Loading: {args.resume}")
        _start_epoch, _best_acc = load_checkpoint(args.resume, model, optimizer=None, map_location=DEVICE)
    else:
        print("[CKPT] No resume checkpoint found; continuing with random init.")

    # ----- Calibration loader -----
    # calib_loader = build_calib_loader(args)

    # def forward_loop(m):
    #     forward_loop_for_calib(m, calib_loader, DEVICE)

    # # ----- Select preset + overrides -----
    # print(f"[ModelOpt] Preset = {args.modelopt_quantize_cfg}")
    # base_cfg = get_modelopt_preset(args.modelopt_quantize_cfg)
    # cfg = base_cfg
    # cfg = apply_custom_overrides(base_cfg)

    # ----- Quantize + Calibrate -----
    t0 = time.time()
    # model = mtq.quantize(model, cfg, forward_loop=forward_loop)  # quantize + calibration :contentReference[oaicite:2]{index=2}
    t1 = time.time()
    print(f"[ModelOpt] quantize+calib done in {t1 - t0:.1f}s")

    # mtq.print_quant_summary(model)  # utile debug scales :contentReference[oaicite:3]{index=3}

    # Optionnel mais recommandé si TensorRT se plaint d'échelles
    # check_quantizers(model)

    # phase QAT
    # print("\n🚀 Démarrage de la phase QAT (Fine-tuning)...")
    # model = enable_qat_learning(model)
    # model.train()
    # # Configuration de l'optimiseur pour le QAT (LR très faible)
    # qat_lr = 1e-5  # Souvent 10x plus petit que le train initial
    # opt = torch.optim.AdamW(model.parameters(), lr=qat_lr, weight_decay=args.weight_decay)
    # class_weights = torch.tensor([0.5, 0.5, 1.0, 1.0, 1.0, 1.2, 1.0], device=DEVICE)

    # # On réutilise vos fonctions de dataset du code de train
    # root = Path(args.data_root)
    # subdirs = sorted([d for d in root.iterdir() if d.is_dir()])
    # start_epoch=0
    # Boucle QAT courte (ex: 1 à 5 époques suffisent souvent)
    # for ep in range(start_epoch, args.epochs):
    #     print(f"\n===== Epoch {ep+1}/{args.epochs} =====")
    #     for sd in subdirs:
    #         ds_tr = ImplicitOccDataset(str(sd), split="train", seed=args.seed)
    #         ds_va = ImplicitOccDataset(str(sd), split="val", seed=args.seed)
    #         dl_tr = DataLoader(ds_tr, batch_size=args.batch_size, shuffle=True, num_workers=args.num_workers, collate_fn=collate_implicit)
    #         dl_va = DataLoader(ds_va, batch_size=args.batch_size, shuffle=False, num_workers=args.num_workers, collate_fn=collate_implicit)

    #         train_loss = train_one_epoch_on_loader(model, dl_tr, opt, DEVICE, args, class_weights)
    #         val_loss, val_acc = evaluate_on_loader(model, dl_va, DEVICE, args, args.out_dir, class_weights, sd.name)
    #         print(f"[{sd.name}] Train Loss: {train_loss:.4f} Val Loss: {val_loss:.4f} Acc: {val_acc*100:.2f}%")



    # ----- Export ONNX Q/DQ -----
    if args.onnx_export_path:
        model.eval()
        export_onnx_qdq(model, args.onnx_export_path, args.onnx_opset)

    print("✅ Done.")


if __name__ == "__main__":
    main()


✅ PE Fourier généré : torch.Size([1, 1024, 512]) (Sigma=8)
[CKPT] Loading: /content/drive/MyDrive/OccMLP/Model_V9/best.ckpt
[ModelOpt] quantize+calib done in 0.0s
[ONNX] Export Q/DQ vers: /content/drive/MyDrive/OccMLP/Model_V9/QAT/surroundocc_int8_q_dq.onnx


/tmp/ipython-input-983092697.py:566: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  denom_y = torch.tensor(h - 1, device=device, dtype=dtype).clamp_min(1.0)
/tmp/ipython-input-983092697.py:566: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  denom_y = torch.tensor(h - 1, device=device, dtype=dtype).clamp_min(1.0)
/tmp/ipython-input-983092697.py:567: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. 

Writing ONNX model to directory: /content/drive/MyDrive/OccMLP/Model_V9/QAT
[ONNX] Export terminé.
✅ Done.


In [ ]:


import torch
import os
import sys

def count_parameters_from_ckpt(ckpt_path):
    print(f"\n📂 Chargement du fichier : {ckpt_path}")

    if not os.path.exists(ckpt_path):
        print("❌ Erreur : Fichier introuvable.")
        return

    # 1. Charger le checkpoint sur CPU (pour ne pas saturer le GPU juste pour compter)
    try:
        checkpoint = torch.load(ckpt_path, map_location="cpu", weights_only=False)
    except Exception as e:
        print(f"❌ Erreur lors du chargement : {e}")
        return

    # 2. Trouver le dictionnaire de poids (state_dict)
    # PyTorch Lightning met souvent les poids sous la clé "state_dict"
    if isinstance(checkpoint, dict):
        if "state_dict" in checkpoint:
            print("ℹ️  Format détecté : PyTorch Lightning / Dict standard")
            state_dict = checkpoint["state_dict"]
        elif "model_state_dict" in checkpoint:
            print("ℹ️  Format détecté : PyTorch Standard")
            state_dict = checkpoint["model_state_dict"]
        elif "model" in checkpoint:
            state_dict = checkpoint["model"]
        else:
            # Parfois on sauvegarde juste le state_dict direct
            print("ℹ️  Format détecté : State Dict Brut")
            state_dict = checkpoint
    else:
        print("❌ Erreur : Le format du checkpoint n'est pas un dictionnaire standard.")
        return

    # 3. Compter les paramètres
    total_params = 0
    total_bytes = 0

    print("\n🔍 Analyse des couches...")

    for key, tensor in state_dict.items():
        # On ignore les buffers de BatchNorm (running_mean, running_var, num_batches_tracked)
        # pour avoir le vrai nombre de paramètres apprenables/fixes.
        if "running_mean" in key or "running_var" in key or "num_batches_tracked" in key:
            continue

        # tensor.numel() donne le nombre total d'éléments (ex: 3x3x3 = 27)
        param_count = tensor.numel()
        total_params += param_count

        # Estimation de la taille en RAM (souvent float32 = 4 octets, float16 = 2 octets)
        elem_size = tensor.element_size()
        total_bytes += param_count * elem_size

    # 4. Affichage joli
    print("-" * 40)
    print(f"🏆 RÉSULTAT POUR : {os.path.basename(ckpt_path)}")
    print("-" * 40)

    # Formatage : 1 200 000 -> 1.20 M
    if total_params >= 1_000_000:
        print(f"📊 Nombre de Paramètres : {total_params / 1_000_000:.2f} M (Millions)")
    elif total_params >= 1_000:
        print(f"📊 Nombre de Paramètres : {total_params / 1_000:.2f} k (Milliers)")
    else:
        print(f"📊 Nombre de Paramètres : {total_params}")

    print(f"💾 Poids estimé en VRAM : {total_bytes / (1024**2):.2f} MB")
    print("-" * 40)

# --- Zone d'utilisation ---
if __name__ == "__main__":
    # Remplace ce chemin par le tien si tu ne veux pas utiliser la ligne de commande
    default_path = "/content/drive/MyDrive/OccMLP/Model_V9/best.ckpt"

    if len(sys.argv) > 1:
        path = default_path
    else:
        path = input(f"Entrez le chemin du .ckpt (Par défaut '{default_path}') : ").strip()
        if path == "":
            path = default_path

    count_parameters_from_ckpt(path)


📂 Chargement du fichier : /content/drive/MyDrive/OccMLP/Model_V9/best.ckpt

🔍 Analyse des couches...
----------------------------------------
🏆 RÉSULTAT POUR : best.ckpt
----------------------------------------
📊 Nombre de Paramètres : 43.44 M (Millions)
💾 Poids estimé en VRAM : 165.70 MB
----------------------------------------


In [ ]:
import os
import time
import numpy as np
import torch
import tensorrt as trt
from torch.utils.data import DataLoader, Subset
from pathlib import Path

# --- CONFIGURATION LOGGING ---
TRT_LOGGER = trt.Logger(trt.Logger.INFO)

class TRTBenchmarkTorch:
    def __init__(self, onnx_path, engine_path):
        self.onnx_path = onnx_path
        self.engine_path = engine_path
        self.runtime = trt.Runtime(TRT_LOGGER)
        self.engine = None
        self.context = None
        self.tensors = {}  # Stockage des tenseurs GPU PyTorch
        self.input_names = []
        self.output_names = []

    def build_engine(self):
        """Charge ou compile l'engine TensorRT"""
        if os.path.exists(self.engine_path):
            print(f"[INFO] Chargement de l'engine : {self.engine_path}")
            with open(self.engine_path, "rb") as f:
                self.engine = self.runtime.deserialize_cuda_engine(f.read())
        else:
            print(f"[INFO] Compilation de l'engine (INT8 + FP16)...")
            builder = trt.Builder(TRT_LOGGER)
            network = builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH))
            parser = trt.OnnxParser(network, TRT_LOGGER)
            config = builder.create_builder_config()

            config.set_flag(trt.BuilderFlag.INT8)
            config.set_flag(trt.BuilderFlag.FP16)
            config.set_flag(trt.BuilderFlag.TF32)
            config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, 1 << 30) # 1GB

            with open(self.onnx_path, "rb") as model:
                if not parser.parse(model.read()):
                    raise RuntimeError("Erreur lors du parsing ONNX")

            plan = builder.build_serialized_network(network, config)
            self.engine = self.runtime.deserialize_cuda_engine(plan)
            with open(self.engine_path, "wb") as f:
                f.write(plan)

        self.context = self.engine.create_execution_context()
        self._allocate_torch_buffers()

    def _allocate_torch_buffers(self):
        """Lien direct entre TensorRT et la mémoire GPU de PyTorch"""
        for i in range(self.engine.num_io_tensors):
            name = self.engine.get_tensor_name(i)
            dtype = self.engine.get_tensor_dtype(name)
            shape = self.engine.get_tensor_shape(name)

            # Conversion Type TRT -> Torch
            torch_dtype = torch.float32 if dtype == trt.float32 else torch.float16
            if dtype == trt.int32: torch_dtype = torch.int32

            # Allocation sur le GPU
            t = torch.empty(tuple(shape), dtype=torch_dtype, device="cuda")
            self.tensors[name] = t

            # Enregistrement des noms pour l'ordre d'inférence
            mode = self.engine.get_tensor_mode(name)
            if mode == trt.TensorIOMode.INPUT:
                self.input_names.append(name)
            else:
                self.output_names.append(name)

            # Liaison de l'adresse mémoire
            self.context.set_tensor_address(name, t.data_ptr())

    def infer(self, input_data_list):
        """Exécution de l'inférence via le stream PyTorch"""
        # 1. Copie des données vers les buffers d'entrée
        for i, name in enumerate(self.input_names):
            self.tensors[name].copy_(input_data_list[i])

        # 2. Exécution asynchrone sur le stream actuel
        stream_handle = torch.cuda.current_stream().cuda_stream
        self.context.execute_async_v3(stream_handle)

        # 3. Retourne les sorties (Dictionnaire de tenseurs GPU)
        return {name: self.tensors[name] for name in self.output_names}

def run_benchmark_torch(onnx_path, data_root, num_samples=500):
    device = "cuda"
    engine_path = onnx_path.replace(".onnx", ".engine")

    # Initialisation de l'outil
    bench = TRTBenchmarkTorch(onnx_path, engine_path)
    bench.build_engine()

    # Déterminer la taille de points attendue (131072)
    input_pts_name = bench.input_names[1] # On suppose que pts est le 2ème input
    max_pts = bench.engine.get_tensor_shape(input_pts_name)[1]

    # DataLoader
    root = Path(data_root)
    subdirs = sorted([d for d in root.iterdir() if d.is_dir()])
    ds_full = ImplicitOccDataset(str(subdirs[0]), split="val", expect_hw=IMAGE_SIZE)
    ds_bench = Subset(ds_full, list(range(min(num_samples, len(ds_full)))))
    dl = DataLoader(ds_bench, batch_size=1, shuffle=False, collate_fn=collate_implicit)

    latencies = []
    correct = 0
    total = 0

    print(f"🚀 Benchmark lancé sur {len(ds_bench)} frames (Padding: {max_pts})...")

    for i, (imgs, pts, labels, mask, _) in enumerate(dl):
        current_n = pts.shape[1]

        # --- Padding des points ---
        if current_n < max_pts:
            padded_pts = torch.zeros((1, max_pts, 3), dtype=pts.dtype, device=device)
            padded_pts[:, :current_n, :] = pts.to(device)
        else:
            padded_pts = pts[:, :max_pts, :].to(device)

        imgs = imgs.to(device)

        # Inférence
        torch.cuda.synchronize()
        start = time.perf_counter()

        outputs = bench.infer([imgs, padded_pts])

        torch.cuda.synchronize()
        latencies.append(time.perf_counter() - start)

        # --- Traitement des Sorties (Shape [1, 7, 131072]) ---
        out_name = bench.output_names[0]
        logits = outputs[out_name]

        # 1. On retire le padding : [1, 7, current_n]
        logits = logits[:, :, :current_n]

        # 2. Argmax sur les 7 classes : [1, current_n]
        preds = torch.argmax(logits, dim=1)

        # 3. Préparation labels et masque
        m = mask.to(device).bool().view(1, -1)
        l = labels.to(device).int().view(1, -1)

        # 4. Calcul Accuracy
        correct += (preds[m] == l[m]).sum().item()
        total += l[m].numel()

        if (i+1) % 50 == 0:
            print(f"Frame {i+1}/{len(ds_bench)} | Acc intermédiaire: {(correct/total)*100:.2f}%")

    # --- Résultats Finaux ---
    avg_latency = np.mean(latencies) * 1000
    final_acc = (correct / total) * 100

    print("\n" + "="*40)
    print(f"RESULTATS TENSORRT 10")
    print("="*40)
    print(f"Précision (Accuracy) : {final_acc:.2f} %")
    print(f"Latence Moyenne      : {avg_latency:.2f} ms")
    print(f"FPS                  : {1000/avg_latency:.1f}")
    print(f"P99 Latence          : {np.percentile(latencies, 99)*1000:.2f} ms")
    print("="*40)

if __name__ == "__main__":
    # Paramètres à adapter
    ONNX_PATH = "/content/drive/MyDrive/OccMLP/Model_V9/QAT/surroundocc_int8_q_dq.onnx"
    DATA_PATH = "/content/DataSet"

    run_benchmark_torch(ONNX_PATH, DATA_PATH, num_samples=500)

[INFO] Chargement de l'engine : /content/drive/MyDrive/OccMLP/Model_V9/QAT/surroundocc_int8_q_dq.engine
[val] frames: 3686 | root=/content/DataSet/mergeData
🚀 Benchmark lancé sur 500 frames (Padding: 131072)...
Frame 50/500 | Acc intermédiaire: 95.69%
Frame 100/500 | Acc intermédiaire: 94.91%
Frame 150/500 | Acc intermédiaire: 95.04%
Frame 200/500 | Acc intermédiaire: 95.22%
Frame 250/500 | Acc intermédiaire: 95.21%
Frame 300/500 | Acc intermédiaire: 95.38%
Frame 350/500 | Acc intermédiaire: 95.48%
Frame 400/500 | Acc intermédiaire: 95.55%
Frame 450/500 | Acc intermédiaire: 95.57%
Frame 500/500 | Acc intermédiaire: 95.64%

RESULTATS TENSORRT 10
Précision (Accuracy) : 95.64 %
Latence Moyenne      : 3.44 ms
FPS                  : 290.4
P99 Latence          : 3.72 ms


In [ ]:
!pip install tensorrt-cu12

  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.9/879.9 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 55.3 MB/s eta 0:00:00
  Created wheel for tensorrt-cu12: filename=tensorrt_cu12-10.14.1.48.post1-py2.py3-none-any.whl size=18166 sha256=4dc2a1b9f99e9ab2c373c37d90ff250d7fb725338f9ff4e61ce1992b129daac6
  Stored in directory: /root/.cache/pip/wheels/41/3b/14/3f0a09edf94b2b99960813c28e648fa36fe47819d62b9a6c84
  Created wheel for tensorrt_cu12_libs: filename=tensorrt_cu12_libs-10.14.1.48.post1-py2.py3-none-manylinux_2_28_x86_64.whl size=3960385540 sha256=0a71baa5f774b843e3b5d1e3d7cb7912c4b5b15eff79c3c10f6728da5251fb18
  Stored in directory: /root/.cache/pip/wheels/f2/c5/ef/0f9de5dd64bab25cabfff0cacd8958d14489f71d92d9602c4c
Successfully built tensorrt-cu12 tensorrt_cu12_libs
  Attempt